In [0]:
import tensorflow
import copy
import torch
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd 
import os

# Download dataset

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls /content/gdrive/"My Drive"/diagnostics_dataset/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
CPRAG-34.tsv		NEG-88-NAT.tsv	 ROLE-88.tsv	    RUS-NEG-30-NAT.xlsx
CPRAG-explanations.tsv	NEG-88-SIMP.tsv  RUS-CPRAG-50.xlsx  RUS-ROLE-24.xlsx


In [0]:
!mkdir datasets

#CPRAG-34

In [0]:
df_CPRAG34 = pd.read_csv(os.path.join("datasets", "CPRAG-34.tsv"),sep='\t')
df_CPRAG34.head()

,item,context_s1,context_s2,expected,within_category,between_category,constraint
0,0,"“Checkmate,” Rosaline announced with glee.",She was getting to be really good at,chess,monopoly,football,H
1,1,Justin put a second house on Park Place.,He and his sister often spent hours playing,monopoly,chess,baseball,H
2,2,He caught the pass and scored another touchdown.,There was nothing he enjoyed more than a good ...,football,baseball,monopoly,H
3,3,Rich couldn’t count the number of Yankees game...,They both shared a lifelong interest in,baseball,football,chess,H
4,4,Getting both himself and his car to work on th...,Every morning he drove for a few minutes and t...,ferry,canoe,plane,L


In [0]:
CLS = "[CLS] "
SEP = " [SEP] "
MASK = " [MASK]"

In [0]:
def add_special_tokens(context1, context2):
  context2 = context2 + MASK
  context1 = CLS + context1 + SEP
  context2 = CLS + context2 + " ." + SEP
  return context1, context2

In [0]:
for i in range(0, df_CPRAG34.shape[0]):
  df_CPRAG34.at[i, "context_s1"], df_CPRAG34.at[i, "context_s2"] = add_special_tokens(df_CPRAG34.iloc[i]["context_s1"], df_CPRAG34.iloc[i]["context_s2"])
  print("{} | {}".format(df_CPRAG34.iloc[i]["context_s1"], df_CPRAG34.iloc[i]["context_s2"]))

[CLS] “Checkmate,” Rosaline announced with glee. [SEP]  | [CLS] She was getting to be really good at [MASK] . [SEP] 
[CLS] Justin put a second house on Park Place. [SEP]  | [CLS] He and his sister often spent hours playing [MASK] . [SEP] 
[CLS] He caught the pass and scored another touchdown. [SEP]  | [CLS] There was nothing he enjoyed more than a good game of [MASK] . [SEP] 
[CLS] Rich couldn’t count the number of Yankees games he had seen with his father. [SEP]  | [CLS] They both shared a lifelong interest in [MASK] . [SEP] 
[CLS] Getting both himself and his car to work on the neighboring island was time-consuming. [SEP]  | [CLS] Every morning he drove for a few minutes and then boarded the [MASK] . [SEP] 
[CLS] The patient was in critical condition and the ambulance wouldn’t be fast enough. [SEP]  | [CLS] They decided they would have to use the [MASK] . [SEP] 
[CLS] Amy was very anxious about traveling abroad for the first time. [SEP]  | [CLS] She felt surprisingly better, however,

In [0]:
def predict_word(context1, context2, tokenizer, model, k=1):
  full_context = context1 + context2
  tokenized_text = tokenizer.tokenize(full_context)
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

  segments_ids = [0] * len(tokenized_text)
  #print(tokenized_text)
  masked_index = tokenized_text.index('[MASK]') 

  predictions = model.call(np.array([indexed_tokens]))
  predicted_index = np.argmax(predictions[0][0,masked_index,:])
  predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
  res = np.argsort(predictions[0][0,masked_index,:])[::-1][:k]
  print(res)
  return res

In [0]:
def check_predicted_words(predicted_words, expected_word):
  for predicted_word in predicted_words:
    if predicted_word == expected_word:
      return True
  return False 

In [0]:
def predict_on_CPRAG34(tokenizer, model, df_CPRAG34, k=1):
  count = 0
  passed = 0
  failed = 0
  for i in range(0, df_CPRAG34.shape[0]):
    print("{} | {}".format(df_CPRAG34.iloc[i]["context_s1"], df_CPRAG34.iloc[i]["context_s2"]))
    predicted_ids = predict_word(df_CPRAG34.iloc[i]["context_s1"], df_CPRAG34.iloc[i]["context_s2"], tokenizer, model, k)
    predicted_words = [tokenizer.convert_ids_to_tokens([word])[0] for word in predicted_ids]
    predicted_words = [word.replace('▁', '') for word in predicted_words]
    #print('Predictions: {}'.format(predicted_words))
    print('Predictions:')
    for pred in predicted_words:
      print('- {}'.format(pred))
    if check_predicted_words(predicted_words, df_CPRAG34.iloc[i]["expected"]):
      passed += 1
      print("PASS: predicted == expected")
    else:
      failed += 1
      print("FAIL: predicted != expected") 
    count += 1
    print("\n")
  accuracy = (float)(passed / count) 
  print("------------------------------------")
  print("ALL: {} | PASSED: {} | FAILED: {}".format(count, passed, failed))
  print("------------------------------------")
  return round(accuracy, 2)

## Bert 

In [0]:
from transformers import BertTokenizer, TFBertForMaskedLM

weights = 'bert-base-uncased'
bert_tokenizer = BertTokenizer.from_pretrained(weights)
bert_model = TFBertForMaskedLM.from_pretrained(weights)

In [0]:
accuracy = predict_on_CPRAG34(bert_tokenizer, bert_model, df_CPRAG34, 1)
print("ACCURACY = {}".format(accuracy))

[CLS] “Checkmate,” Rosaline announced with glee. [SEP]  | [CLS] She was getting to be really good at [MASK] . [SEP] 
[2009]
Predictions:
- it
FAIL: predicted != expected


[CLS] Justin put a second house on Park Place. [SEP]  | [CLS] He and his sister often spent hours playing [MASK] . [SEP] 
[2362]
Predictions:
- together
FAIL: predicted != expected


[CLS] He caught the pass and scored another touchdown. [SEP]  | [CLS] There was nothing he enjoyed more than a good game of [MASK] . [SEP] 
[2374]
Predictions:
- football
PASS: predicted == expected


[CLS] Rich couldn’t count the number of Yankees games he had seen with his father. [SEP]  | [CLS] They both shared a lifelong interest in [MASK] . [SEP] 
[3598]
Predictions:
- baseball
PASS: predicted == expected


[CLS] Getting both himself and his car to work on the neighboring island was time-consuming. [SEP]  | [CLS] Every morning he drove for a few minutes and then boarded the [MASK] . [SEP] 
[7115]
Predictions:
- ferry
PASS: predicted

In [0]:
accuracy = predict_on_CPRAG34(bert_tokenizer, bert_model, df_CPRAG34, 3)
print("ACCURACY = {}".format(accuracy))

[CLS] “Checkmate,” Rosaline announced with glee. [SEP]  | [CLS] She was getting to be really good at [MASK] . [SEP] 
[2009 2023 2008]
Predictions:
- it
- this
- that
FAIL: predicted != expected


[CLS] Justin put a second house on Park Place. [SEP]  | [CLS] He and his sister often spent hours playing [MASK] . [SEP] 
[2362 4770 2648]
Predictions:
- together
- pool
- outside
FAIL: predicted != expected


[CLS] He caught the pass and scored another touchdown. [SEP]  | [CLS] There was nothing he enjoyed more than a good game of [MASK] . [SEP] 
[2374 3455 3598]
Predictions:
- football
- basketball
- baseball
PASS: predicted == expected


[CLS] Rich couldn’t count the number of Yankees games he had seen with his father. [SEP]  | [CLS] They both shared a lifelong interest in [MASK] . [SEP] 
[3598 2998 2374]
Predictions:
- baseball
- sports
- football
PASS: predicted == expected


[CLS] Getting both himself and his car to work on the neighboring island was time-consuming. [SEP]  | [CLS] Every 

In [0]:
accuracy = predict_on_CPRAG34(bert_tokenizer, bert_model, df_CPRAG34, 5)
print("ACCURACY = {}".format(accuracy))

[CLS] “Checkmate,” Rosaline announced with glee. [SEP]  | [CLS] She was getting to be really good at [MASK] . [SEP] 
[2009 2023 2008 7433 8785]
Predictions:
- it
- this
- that
- chess
- math
PASS: predicted == expected


[CLS] Justin put a second house on Park Place. [SEP]  | [CLS] He and his sister often spent hours playing [MASK] . [SEP] 
[2362 4770 2648 5329 4715]
Predictions:
- together
- pool
- outside
- cards
- soccer
FAIL: predicted != expected


[CLS] He caught the pass and scored another touchdown. [SEP]  | [CLS] There was nothing he enjoyed more than a good game of [MASK] . [SEP] 
[2374 3455 3598 2009 5439]
Predictions:
- football
- basketball
- baseball
- it
- golf
PASS: predicted == expected


[CLS] Rich couldn’t count the number of Yankees games he had seen with his father. [SEP]  | [CLS] They both shared a lifelong interest in [MASK] . [SEP] 
[ 3598  2998  2374  3455 12585]
Predictions:
- baseball
- sports
- football
- basketball
- softball
PASS: predicted == expected


[

In [0]:
accuracy = predict_on_CPRAG34(bert_tokenizer, bert_model, df_CPRAG34, 10)
print("ACCURACY = {}".format(accuracy))

[CLS] “Checkmate,” Rosaline announced with glee. [SEP]  | [CLS] She was getting to be really good at [MASK] . [SEP] 
[2009 2023 2008 7433 8785 3554 4688 3752 3772 2673]
Predictions:
- it
- this
- that
- chess
- math
- fighting
- lying
- reading
- acting
- everything
PASS: predicted == expected


[CLS] Justin put a second house on Park Place. [SEP]  | [CLS] He and his sister often spent hours playing [MASK] . [SEP] 
[2362 4770 2648 5329 4715 5439 2374 2045 5093 3598]
Predictions:
- together
- pool
- outside
- cards
- soccer
- golf
- football
- there
- tennis
- baseball
FAIL: predicted != expected


[CLS] He caught the pass and scored another touchdown. [SEP]  | [CLS] There was nothing he enjoyed more than a good game of [MASK] . [SEP] 
[2374 3455 3598 2009 5439 2770 4458 4715 3639 3608]
Predictions:
- football
- basketball
- baseball
- it
- golf
- running
- passing
- soccer
- defense
- ball
PASS: predicted == expected


[CLS] Rich couldn’t count the number of Yankees games he had seen w

## DistilBert (distilbert-base-cased)

In [0]:
from transformers import DistilBertTokenizer, TFDistilBertForMaskedLM

weights = 'distilbert-base-cased'
distilbert_tokenizer = DistilBertTokenizer.from_pretrained(weights)
distilbert_model = TFDistilBertForMaskedLM.from_pretrained(weights)

k = 1

In [0]:
accuracy = predict_on_CPRAG34(distilbert_tokenizer, distilbert_model, df_CPRAG34, 1)
print("ACCURACY = {}".format(accuracy))

[CLS] “Checkmate,” Rosaline announced with glee. [SEP]  | [CLS] She was getting to be really good at [MASK] . [SEP] 
[5923]
Predictions:
- dancing
FAIL: predicted != expected


[CLS] Justin put a second house on Park Place. [SEP]  | [CLS] He and his sister often spent hours playing [MASK] . [SEP] 
[5538]
Predictions:
- tennis
FAIL: predicted != expected


[CLS] He caught the pass and scored another touchdown. [SEP]  | [CLS] There was nothing he enjoyed more than a good game of [MASK] . [SEP] 
[1709]
Predictions:
- football
PASS: predicted == expected


[CLS] Rich couldn’t count the number of Yankees games he had seen with his father. [SEP]  | [CLS] They both shared a lifelong interest in [MASK] . [SEP] 
[3866]
Predictions:
- baseball
PASS: predicted == expected


[CLS] Getting both himself and his car to work on the neighboring island was time-consuming. [SEP]  | [CLS] Every morning he drove for a few minutes and then boarded the [MASK] . [SEP] 
[4261]
Predictions:
- plane
FAIL: predic

k = 3 

In [0]:
accuracy = predict_on_CPRAG34(distilbert_tokenizer, distilbert_model, df_CPRAG34, 3)
print("ACCURACY = {}".format(accuracy))

[CLS] “Checkmate,” Rosaline announced with glee. [SEP]  | [CLS] She was getting to be really good at [MASK] . [SEP] 
[5923 4241 3176]
Predictions:
- dancing
- singing
- acting
FAIL: predicted != expected


[CLS] Justin put a second house on Park Place. [SEP]  | [CLS] He and his sister often spent hours playing [MASK] . [SEP] 
[ 5538  7135 15772]
Predictions:
- tennis
- golf
- poker
FAIL: predicted != expected


[CLS] He caught the pass and scored another touchdown. [SEP]  | [CLS] There was nothing he enjoyed more than a good game of [MASK] . [SEP] 
[1709 3163 1505]
Predictions:
- football
- basketball
- play
PASS: predicted == expected


[CLS] Rich couldn’t count the number of Yankees games he had seen with his father. [SEP]  | [CLS] They both shared a lifelong interest in [MASK] . [SEP] 
[3866 1709 3163]
Predictions:
- baseball
- football
- basketball
PASS: predicted == expected


[CLS] Getting both himself and his car to work on the neighboring island was time-consuming. [SEP]  | [CL

k = 5

In [0]:
accuracy = predict_on_CPRAG34(distilbert_tokenizer, distilbert_model, df_CPRAG34, 5)
print("ACCURACY = {}".format(accuracy))

[CLS] “Checkmate,” Rosaline announced with glee. [SEP]  | [CLS] She was getting to be really good at [MASK] . [SEP] 
[ 5923  4241  3176 15744  1122]
Predictions:
- dancing
- singing
- acting
- songwriting
- it
FAIL: predicted != expected


[CLS] Justin put a second house on Park Place. [SEP]  | [CLS] He and his sister often spent hours playing [MASK] . [SEP] 
[ 5538  7135 15772  1487  4802]
Predictions:
- tennis
- golf
- poker
- together
- cards
FAIL: predicted != expected


[CLS] He caught the pass and scored another touchdown. [SEP]  | [CLS] There was nothing he enjoyed more than a good game of [MASK] . [SEP] 
[1709 3163 1505 2415 3744]
Predictions:
- football
- basketball
- play
- practice
- passing
PASS: predicted == expected


[CLS] Rich couldn’t count the number of Yankees games he had seen with his father. [SEP]  | [CLS] They both shared a lifelong interest in [MASK] . [SEP] 
[ 3866  1709  3163  2865 10568]
Predictions:
- baseball
- football
- basketball
- sports
- boxing
PASS: 

k = 10

In [0]:
accuracy = predict_on_CPRAG34(distilbert_tokenizer, distilbert_model, df_CPRAG34, 10)
print("ACCURACY = {}".format(accuracy))

[CLS] “Checkmate,” Rosaline announced with glee. [SEP]  | [CLS] She was getting to be really good at [MASK] . [SEP] 
[ 5923  4241  3176 15744  1122 15772  8739 25514  7325  1390]
Predictions:
- dancing
- singing
- acting
- songwriting
- it
- poker
- cooking
- storytelling
- wrestling
- music
FAIL: predicted != expected


[CLS] Justin put a second house on Park Place. [SEP]  | [CLS] He and his sister often spent hours playing [MASK] . [SEP] 
[ 5538  7135 15772  1487  4802 10924  3267  3163 17607 23178]
Predictions:
- tennis
- golf
- poker
- together
- cards
- chess
- piano
- basketball
- softball
- outdoors
FAIL: predicted != expected


[CLS] He caught the pass and scored another touchdown. [SEP]  | [CLS] There was nothing he enjoyed more than a good game of [MASK] . [SEP] 
[ 1709  3163  1505  2415  3744 10027  3948  2168  6920  3866]
Predictions:
- football
- basketball
- play
- practice
- passing
- offense
- defense
- action
- luck
- baseball
PASS: predicted == expected


[CLS] Rich c

## Albert (albert-base-v2) 

In [0]:
from transformers import AlbertTokenizer, TFAlbertForMaskedLM

weights = 'albert-base-v2'
albert_tokenizer = AlbertTokenizer.from_pretrained(weights)
albert_model = TFAlbertForMaskedLM.from_pretrained(weights)

k = 1

In [0]:
accuracy = predict_on_CPRAG34(albert_tokenizer, albert_model, df_CPRAG34, 1)
print("ACCURACY = {}".format(accuracy))

[CLS] “Checkmate,” Rosaline announced with glee. [SEP]  | [CLS] She was getting to be really good at [MASK] . [SEP] 
[29833]
Predictions:
- evalle
FAIL: predicted != expected


[CLS] Justin put a second house on Park Place. [SEP]  | [CLS] He and his sister often spent hours playing [MASK] . [SEP] 
[6168]
Predictions:
- chess
FAIL: predicted != expected


[CLS] He caught the pass and scored another touchdown. [SEP]  | [CLS] There was nothing he enjoyed more than a good game of [MASK] . [SEP] 
[435]
Predictions:
- football
PASS: predicted == expected


[CLS] Rich couldn’t count the number of Yankees games he had seen with his father. [SEP]  | [CLS] They both shared a lifelong interest in [MASK] . [SEP] 
[1845]
Predictions:
- baseball
PASS: predicted == expected


[CLS] Getting both himself and his car to work on the neighboring island was time-consuming. [SEP]  | [CLS] Every morning he drove for a few minutes and then boarded the [MASK] . [SEP] 
[5901]
Predictions:
- ferry
PASS: predicte

k = 3

In [0]:
accuracy = predict_on_CPRAG34(albert_tokenizer, albert_model, df_CPRAG34, 3)
print("ACCURACY = {}".format(accuracy))

[CLS] “Checkmate,” Rosaline announced with glee. [SEP]  | [CLS] She was getting to be really good at [MASK] . [SEP] 
[29833 28153    32]
Predictions:
- evalle
- joyah
- it
FAIL: predicted != expected


[CLS] Justin put a second house on Park Place. [SEP]  | [CLS] He and his sister often spent hours playing [MASK] . [SEP] 
[ 6168  6782 27563]
Predictions:
- chess
- volleyball
- bingo
FAIL: predicted != expected


[CLS] He caught the pass and scored another touchdown. [SEP]  | [CLS] There was nothing he enjoyed more than a good game of [MASK] . [SEP] 
[ 435 2184 1652]
Predictions:
- football
- hockey
- basketball
PASS: predicted == expected


[CLS] Rich couldn’t count the number of Yankees games he had seen with his father. [SEP]  | [CLS] They both shared a lifelong interest in [MASK] . [SEP] 
[1845 1652 6168]
Predictions:
- baseball
- basketball
- chess
PASS: predicted == expected


[CLS] Getting both himself and his car to work on the neighboring island was time-consuming. [SEP]  | [CL

k = 5

In [0]:
accuracy = predict_on_CPRAG34(albert_tokenizer, albert_model, df_CPRAG34, 5)
print("ACCURACY = {}".format(accuracy))

[CLS] “Checkmate,” Rosaline announced with glee. [SEP]  | [CLS] She was getting to be really good at [MASK] . [SEP] 
[29833 28153    32 25266    48]
Predictions:
- evalle
- joyah
- it
- typing
- this
FAIL: predicted != expected


[CLS] Justin put a second house on Park Place. [SEP]  | [CLS] He and his sister often spent hours playing [MASK] . [SEP] 
[ 6168  6782 27563   466  3637]
Predictions:
- chess
- volleyball
- bingo
- games
- tennis
FAIL: predicted != expected


[CLS] He caught the pass and scored another touchdown. [SEP]  | [CLS] There was nothing he enjoyed more than a good game of [MASK] . [SEP] 
[  435  2184  1652 29617  3173]
Predictions:
- football
- hockey
- basketball
- futbol
- soccer
PASS: predicted == expected


[CLS] Rich couldn’t count the number of Yankees games he had seen with his father. [SEP]  | [CLS] They both shared a lifelong interest in [MASK] . [SEP] 
[1845 1652 6168 2184 3173]
Predictions:
- baseball
- basketball
- chess
- hockey
- soccer
PASS: predicted =

In [0]:
accuracy = predict_on_CPRAG34(albert_tokenizer, albert_model, df_CPRAG34, 10)
print("ACCURACY = {}".format(accuracy))

[CLS] “Checkmate,” Rosaline announced with glee. [SEP]  | [CLS] She was getting to be really good at [MASK] . [SEP] 
[29833 28153    32 25266    48    30   845   301    36 27898]
Predictions:
- evalle
- joyah
- it
- typing
- this
- that
- doing
- something
- her
- vimes
FAIL: predicted != expected


[CLS] Justin put a second house on Park Place. [SEP]  | [CLS] He and his sister often spent hours playing [MASK] . [SEP] 
[ 6168  6782 27563   466  3637  3173   429  4092  4034 14401]
Predictions:
- chess
- volleyball
- bingo
- games
- tennis
- soccer
- together
- cards
- golf
- softball
FAIL: predicted != expected


[CLS] He caught the pass and scored another touchdown. [SEP]  | [CLS] There was nothing he enjoyed more than a good game of [MASK] . [SEP] 
[  435  2184  1652 29617  3173  4034  1845  6168 15695 11228]
Predictions:
- football
- hockey
- basketball
- futbol
- soccer
- golf
- baseball
- chess
- lacrosse
- kicking
PASS: predicted == expected


[CLS] Rich couldn’t count the number 

## Albert (albert-large-v2)

In [0]:
from transformers import AlbertTokenizer, TFAlbertForMaskedLM

weights = 'albert-large-v2'
albert_large_tokenizer = AlbertTokenizer.from_pretrained(weights)
albert_large_model = TFAlbertForMaskedLM.from_pretrained(weights)

k = 1

In [0]:
accuracy = predict_on_CPRAG34(albert_large_tokenizer, albert_large_model, df_CPRAG34, 1)
print("ACCURACY = {}".format(accuracy))

[CLS] “Checkmate,” Rosaline announced with glee. [SEP]  | [CLS] She was getting to be really good at [MASK] . [SEP] 
[48]
Predictions:
- this
FAIL: predicted != expected


[CLS] Justin put a second house on Park Place. [SEP]  | [CLS] He and his sister often spent hours playing [MASK] . [SEP] 
[1845]
Predictions:
- baseball
FAIL: predicted != expected


[CLS] He caught the pass and scored another touchdown. [SEP]  | [CLS] There was nothing he enjoyed more than a good game of [MASK] . [SEP] 
[1845]
Predictions:
- baseball
FAIL: predicted != expected


[CLS] Rich couldn’t count the number of Yankees games he had seen with his father. [SEP]  | [CLS] They both shared a lifelong interest in [MASK] . [SEP] 
[1845]
Predictions:
- baseball
PASS: predicted == expected


[CLS] Getting both himself and his car to work on the neighboring island was time-consuming. [SEP]  | [CLS] Every morning he drove for a few minutes and then boarded the [MASK] . [SEP] 
[1683]
Predictions:
- bus
FAIL: predicted !

k = 3

In [0]:
accuracy = predict_on_CPRAG34(albert_large_tokenizer, albert_large_model, df_CPRAG34, 3)
print("ACCURACY = {}".format(accuracy))

[CLS] “Checkmate,” Rosaline announced with glee. [SEP]  | [CLS] She was getting to be really good at [MASK] . [SEP] 
[   48    32 25266]
Predictions:
- this
- it
- typing
FAIL: predicted != expected


[CLS] Justin put a second house on Park Place. [SEP]  | [CLS] He and his sister often spent hours playing [MASK] . [SEP] 
[1845 3637 3173]
Predictions:
- baseball
- tennis
- soccer
FAIL: predicted != expected


[CLS] He caught the pass and scored another touchdown. [SEP]  | [CLS] There was nothing he enjoyed more than a good game of [MASK] . [SEP] 
[1845  435 1652]
Predictions:
- baseball
- football
- basketball
PASS: predicted == expected


[CLS] Rich couldn’t count the number of Yankees games he had seen with his father. [SEP]  | [CLS] They both shared a lifelong interest in [MASK] . [SEP] 
[1845 1652 1059]
Predictions:
- baseball
- basketball
- sports
PASS: predicted == expected


[CLS] Getting both himself and his car to work on the neighboring island was time-consuming. [SEP]  | [CLS

k = 5

In [0]:
accuracy = predict_on_CPRAG34(albert_large_tokenizer, albert_large_model, df_CPRAG34, 5)
print("ACCURACY = {}".format(accuracy))

[CLS] “Checkmate,” Rosaline announced with glee. [SEP]  | [CLS] She was getting to be really good at [MASK] . [SEP] 
[   48    32 25266   791 13406]
Predictions:
- this
- it
- typing
- playing
- pretending
FAIL: predicted != expected


[CLS] Justin put a second house on Park Place. [SEP]  | [CLS] He and his sister often spent hours playing [MASK] . [SEP] 
[1845 3637 3173 6782 4034]
Predictions:
- baseball
- tennis
- soccer
- volleyball
- golf
FAIL: predicted != expected


[CLS] He caught the pass and scored another touchdown. [SEP]  | [CLS] There was nothing he enjoyed more than a good game of [MASK] . [SEP] 
[1845  435 1652 3173 4034]
Predictions:
- baseball
- football
- basketball
- soccer
- golf
PASS: predicted == expected


[CLS] Rich couldn’t count the number of Yankees games he had seen with his father. [SEP]  | [CLS] They both shared a lifelong interest in [MASK] . [SEP] 
[1845 1652 1059 2184  435]
Predictions:
- baseball
- basketball
- sports
- hockey
- football
PASS: predicted

k = 10

In [0]:
accuracy = predict_on_CPRAG34(albert_large_tokenizer, albert_large_model, df_CPRAG34, 10)
print("ACCURACY = {}".format(accuracy))

[CLS] “Checkmate,” Rosaline announced with glee. [SEP]  | [CLS] She was getting to be really good at [MASK] . [SEP] 
[   48    32 25266   791 13406    30 25128   602   301  6168]
Predictions:
- this
- it
- typing
- playing
- pretending
- that
- spying
- anything
- something
- chess
PASS: predicted == expected


[CLS] Justin put a second house on Park Place. [SEP]  | [CLS] He and his sister often spent hours playing [MASK] . [SEP] 
[ 1845  3637  3173  6782  4034  1652 16953   435 14401  6168]
Predictions:
- baseball
- tennis
- soccer
- volleyball
- golf
- basketball
- badminton
- football
- softball
- chess
FAIL: predicted != expected


[CLS] He caught the pass and scored another touchdown. [SEP]  | [CLS] There was nothing he enjoyed more than a good game of [MASK] . [SEP] 
[ 1845   435  1652  3173  4034  2184 15695 12453 14401  2362]
Predictions:
- baseball
- football
- basketball
- soccer
- golf
- hockey
- lacrosse
- poker
- softball
- rugby
PASS: predicted == expected


[CLS] Rich co

# ROLE-88

In [0]:
df_ROLE88 = pd.read_csv(os.path.join("datasets", "ROLE-88.tsv"), sep='\t')
df_ROLE88.head()

,item,context,expected,exp_cloze,target,tgt_cloze,tgt_cloze(strict)
0,61-b,The librarian documented which journalist the ...,avoided|hated|read|met,0.066667,interviewed,0.066667,0.000000
1,61-a,The librarian documented which celebrities the...,interviewed,0.533333,interviewed,0.533333,0.533333
2,62-b,the old widower remembered which ghost the vil...,seen,0.600000,haunted,0.000000,0.000000
3,62-a,the old widower remembered which villager the ...,haunted,0.500000,haunted,0.500000,0.500000
4,63-b,the firefighter reported which paramedic the v...,thanked,0.200000,saved,0.166667,0.000000


In [0]:
def add_special_tokens(context):
  context = context + MASK
  context = CLS + context + " ." + SEP
  return context

In [0]:
for i in range(0, df_ROLE88.shape[0]):
  df_ROLE88.at[i, "context"] = add_special_tokens(df_ROLE88.iloc[i]["context"])
  print("{}".format(df_ROLE88.iloc[i]["context"]))

[CLS] The librarian documented which journalist the celebrities had  [MASK] . [SEP] 
[CLS] The librarian documented which celebrities the journalist had  [MASK] . [SEP] 
[CLS] the old widower remembered which ghost the villager had  [MASK] . [SEP] 
[CLS] the old widower remembered which villager the ghost had  [MASK] . [SEP] 
[CLS] the firefighter reported which paramedic the victim had  [MASK] . [SEP] 
[CLS] the firefighter reported which victim the paramedic had  [MASK] . [SEP] 
[CLS] the historian documented which assassin the prince had  [MASK] . [SEP] 
[CLS] the historian documented which prince the assassin had  [MASK] . [SEP] 
[CLS] the judo master recognized which assailant the defender had  [MASK] . [SEP] 
[CLS] the judo master recognized which defender the assailant had  [MASK] . [SEP] 
[CLS] the naturalist observed which deer the predators had  [MASK] . [SEP] 
[CLS] the naturalist observed which predators the deer had  [MASK] . [SEP] 
[CLS] the restaurant owner forgot which 

In [0]:
def predict_word(context, tokenizer, model, k=1):
  tokenized_text = tokenizer.tokenize(context)
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

  segments_ids = [0] * len(tokenized_text)
  #print(tokenized_text)
  masked_index = tokenized_text.index('[MASK]') 

  predictions = model.call(np.array([indexed_tokens]))
  predicted_index = np.argmax(predictions[0][0,masked_index,:])
  predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
  res = np.argsort(predictions[0][0,masked_index,:])[::-1][:k]
  return res

In [0]:
def predict_on_ROLE_88(tokenizer, model, df_ROLE88, k=1):
  count = 0
  passed = 0
  failed = 0
  for i in range(0, df_ROLE88.shape[0]):
    print("{}".format(df_ROLE88.iloc[i]["context"]))
    predicted_ids = predict_word(df_ROLE88.iloc[i]["context"], tokenizer, model, k)
    predicted_words = [tokenizer.convert_ids_to_tokens([word])[0] for word in predicted_ids]
    predicted_words = [word.replace('▁', '') for word in predicted_words]
    #print('Predictions: {}'.format(predicted_words))
    print('Predictions:')
    for pred in predicted_words:
      print('- {}'.format(pred))
    exps = df_ROLE88.iloc[i]["expected"].split("|")

    detected = False
    for exp in exps:
      if check_predicted_words(predicted_words, exp):
        detected = True
        break
    if detected:
      passed += 1
      print("PASS: predicted == expected")
    else:
      failed += 1
      print("FAIL: predicted != expected") 
    if check_predicted_words(predicted_words, df_ROLE88.iloc[i]["target"]):
      print("GOOD: predicted == target")
    print("expected: {}, target: {}".format(df_ROLE88.iloc[i]["expected"], 
                                            df_ROLE88.iloc[i]["target"]))
    count += 1
    print("\n")
  accuracy = (float)(passed / count) 
  print("------------------------------------")
  print("ALL: {} | PASSED: {} | FAILED: {}".format(count, passed, failed))
  print("------------------------------------")
  return round(accuracy, 2)

## Bert

In [0]:
from transformers import BertTokenizer, TFBertForMaskedLM

weights = 'bert-base-uncased'
bert_tokenizer = BertTokenizer.from_pretrained(weights)
bert_model = TFBertForMaskedLM.from_pretrained(weights)

k = 1

In [0]:
accuracy = predict_on_ROLE_88(bert_tokenizer, bert_model, df_ROLE88, 1)
print("ACCURACY = {}".format(accuracy))

[CLS] The librarian documented which journalist the celebrities had  [MASK] . [SEP] 
Predictions:
- chosen
FAIL: predicted != expected
expected: avoided|hated|read|met, target: interviewed


[CLS] The librarian documented which celebrities the journalist had  [MASK] . [SEP] 
Predictions:
- taken
FAIL: predicted != expected
expected: interviewed, target: interviewed


[CLS] the old widower remembered which ghost the villager had  [MASK] . [SEP] 
Predictions:
- mentioned
FAIL: predicted != expected
expected: seen, target: haunted


[CLS] the old widower remembered which villager the ghost had  [MASK] . [SEP] 
Predictions:
- killed
FAIL: predicted != expected
expected: haunted, target: haunted


[CLS] the firefighter reported which paramedic the victim had  [MASK] . [SEP] 
Predictions:
- been
FAIL: predicted != expected
expected: thanked, target: saved


[CLS] the firefighter reported which victim the paramedic had  [MASK] . [SEP] 
Predictions:
- taken
FAIL: predicted != expected
expected

k = 3

In [0]:
accuracy = predict_on_ROLE_88(bert_tokenizer, bert_model, df_ROLE88, 3)
print("ACCURACY = {}".format(accuracy))

[CLS] The librarian documented which journalist the celebrities had  [MASK] . [SEP] 
Predictions:
- chosen
- been
- taken
FAIL: predicted != expected
expected: avoided|hated|read|met, target: interviewed


[CLS] The librarian documented which celebrities the journalist had  [MASK] . [SEP] 
Predictions:
- taken
- written
- used
FAIL: predicted != expected
expected: interviewed, target: interviewed


[CLS] the old widower remembered which ghost the villager had  [MASK] . [SEP] 
Predictions:
- mentioned
- left
- killed
FAIL: predicted != expected
expected: seen, target: haunted


[CLS] the old widower remembered which villager the ghost had  [MASK] . [SEP] 
Predictions:
- killed
- been
- seen
FAIL: predicted != expected
expected: haunted, target: haunted


[CLS] the firefighter reported which paramedic the victim had  [MASK] . [SEP] 
Predictions:
- been
- seen
- killed
FAIL: predicted != expected
expected: thanked, target: saved


[CLS] the firefighter reported which victim the paramedic 

k = 5

In [0]:
accuracy = predict_on_ROLE_88(bert_tokenizer, bert_model, df_ROLE88, 5)
print("ACCURACY = {}".format(accuracy))

[CLS] The librarian documented which journalist the celebrities had  [MASK] . [SEP] 
Predictions:
- chosen
- been
- taken
- used
- interviewed
FAIL: predicted != expected
GOOD: predicted == target
expected: avoided|hated|read|met, target: interviewed


[CLS] The librarian documented which celebrities the journalist had  [MASK] . [SEP] 
Predictions:
- taken
- written
- used
- made
- read
FAIL: predicted != expected
expected: interviewed, target: interviewed


[CLS] the old widower remembered which ghost the villager had  [MASK] . [SEP] 
Predictions:
- mentioned
- left
- killed
- seen
- been
PASS: predicted == expected
expected: seen, target: haunted


[CLS] the old widower remembered which villager the ghost had  [MASK] . [SEP] 
Predictions:
- killed
- been
- seen
- murdered
- struck
FAIL: predicted != expected
expected: haunted, target: haunted


[CLS] the firefighter reported which paramedic the victim had  [MASK] . [SEP] 
Predictions:
- been
- seen
- killed
- called
- beaten
FAIL: pr

k = 10

In [0]:
accuracy = predict_on_ROLE_88(bert_tokenizer, bert_model, df_ROLE88, 10)
print("ACCURACY = {}".format(accuracy))

[CLS] The librarian documented which journalist the celebrities had  [MASK] . [SEP] 
Predictions:
- chosen
- been
- taken
- used
- interviewed
- played
- killed
- hired
- sent
- picked
FAIL: predicted != expected
GOOD: predicted == target
expected: avoided|hated|read|met, target: interviewed


[CLS] The librarian documented which celebrities the journalist had  [MASK] . [SEP] 
Predictions:
- taken
- written
- used
- made
- read
- mentioned
- found
- chosen
- visited
- killed
FAIL: predicted != expected
expected: interviewed, target: interviewed


[CLS] the old widower remembered which ghost the villager had  [MASK] . [SEP] 
Predictions:
- mentioned
- left
- killed
- seen
- been
- heard
- called
- said
- found
- sent
PASS: predicted == expected
expected: seen, target: haunted


[CLS] the old widower remembered which villager the ghost had  [MASK] . [SEP] 
Predictions:
- killed
- been
- seen
- murdered
- struck
- attacked
- mentioned
- visited
- slain
- told
FAIL: predicted != expected
e

## DistilBert (distilbert-base-cased)

In [0]:
from transformers import DistilBertTokenizer, TFDistilBertForMaskedLM

weights = 'distilbert-base-cased'
distilbert_tokenizer = DistilBertTokenizer.from_pretrained(weights)
distilbert_model = TFDistilBertForMaskedLM.from_pretrained(weights)

k = 1

In [0]:
accuracy = predict_on_ROLE_88(distilbert_tokenizer, distilbert_model, df_ROLE88, 1)
print("ACCURACY = {}".format(accuracy))

[CLS] The librarian documented which journalist the celebrities had  [MASK] . [SEP] 
Predictions:
- interviewed
FAIL: predicted != expected
GOOD: predicted == target
expected: avoided|hated|read|met, target: interviewed


[CLS] The librarian documented which celebrities the journalist had  [MASK] . [SEP] 
Predictions:
- interviewed
PASS: predicted == expected
GOOD: predicted == target
expected: interviewed, target: interviewed


[CLS] the old widower remembered which ghost the villager had  [MASK] . [SEP] 
Predictions:
- chosen
FAIL: predicted != expected
expected: seen, target: haunted


[CLS] the old widower remembered which villager the ghost had  [MASK] . [SEP] 
Predictions:
- murdered
FAIL: predicted != expected
expected: haunted, target: haunted


[CLS] the firefighter reported which paramedic the victim had  [MASK] . [SEP] 
Predictions:
- killed
FAIL: predicted != expected
expected: thanked, target: saved


[CLS] the firefighter reported which victim the paramedic had  [MASK] . 

k = 3

In [0]:
accuracy = predict_on_ROLE_88(albert_tokenizer, albert_model, df_ROLE88, 3)
print("ACCURACY = {}".format(accuracy))

[CLS] The librarian documented which journalist the celebrities had  [MASK] . [SEP] 
Predictions:
- interviewed
- contacted
- hired
FAIL: predicted != expected
GOOD: predicted == target
expected: avoided|hated|read|met, target: interviewed


[CLS] The librarian documented which celebrities the journalist had  [MASK] . [SEP] 
Predictions:
- written
- done
- authored
FAIL: predicted != expected
expected: interviewed, target: interviewed


[CLS] the old widower remembered which ghost the villager had  [MASK] . [SEP] 
Predictions:
- summoned
- killed
- slain
FAIL: predicted != expected
expected: seen, target: haunted


[CLS] the old widower remembered which villager the ghost had  [MASK] . [SEP] 
Predictions:
- attacked
- murdered
- killed
FAIL: predicted != expected
expected: haunted, target: haunted


[CLS] the firefighter reported which paramedic the victim had  [MASK] . [SEP] 
Predictions:
- hired
- been
- contacted
FAIL: predicted != expected
expected: thanked, target: saved


[CLS] t

k = 5

In [0]:
accuracy = predict_on_ROLE_88(albert_tokenizer, albert_model, df_ROLE88, 5)
print("ACCURACY = {}".format(accuracy))

[CLS] The librarian documented which journalist the celebrities had  [MASK] . [SEP] 
Predictions:
- interviewed
- contacted
- hired
- consulted
- befriended
FAIL: predicted != expected
GOOD: predicted == target
expected: avoided|hated|read|met, target: interviewed


[CLS] The librarian documented which celebrities the journalist had  [MASK] . [SEP] 
Predictions:
- written
- done
- authored
- addressed
- interviewed
PASS: predicted == expected
GOOD: predicted == target
expected: interviewed, target: interviewed


[CLS] the old widower remembered which ghost the villager had  [MASK] . [SEP] 
Predictions:
- summoned
- killed
- slain
- murdered
- seen
PASS: predicted == expected
expected: seen, target: haunted


[CLS] the old widower remembered which villager the ghost had  [MASK] . [SEP] 
Predictions:
- attacked
- murdered
- killed
- visited
- summoned
FAIL: predicted != expected
expected: haunted, target: haunted


[CLS] the firefighter reported which paramedic the victim had  [MASK] . [

k = 10

In [0]:
accuracy = predict_on_ROLE_88(albert_tokenizer, albert_model, df_ROLE88, 10)
print("ACCURACY = {}".format(accuracy))

[CLS] The librarian documented which journalist the celebrities had  [MASK] . [SEP] 
Predictions:
- interviewed
- contacted
- hired
- consulted
- befriended
- been
- visited
- encountered
- recruited
- employed
FAIL: predicted != expected
GOOD: predicted == target
expected: avoided|hated|read|met, target: interviewed


[CLS] The librarian documented which celebrities the journalist had  [MASK] . [SEP] 
Predictions:
- written
- done
- authored
- addressed
- interviewed
- visited
- taken
- performed
- contacted
- encountered
PASS: predicted == expected
GOOD: predicted == target
expected: interviewed, target: interviewed


[CLS] the old widower remembered which ghost the villager had  [MASK] . [SEP] 
Predictions:
- summoned
- killed
- slain
- murdered
- seen
- encountered
- chased
- attacked
- found
- stabbed
PASS: predicted == expected
expected: seen, target: haunted


[CLS] the old widower remembered which villager the ghost had  [MASK] . [SEP] 
Predictions:
- attacked
- murdered
- kill

## Albert (albert-base-v2) 

In [0]:
from transformers import AlbertTokenizer, TFAlbertForMaskedLM

weights = 'albert-base-v2'
albert_tokenizer = AlbertTokenizer.from_pretrained(weights)
albert_model = TFAlbertForMaskedLM.from_pretrained(weights)

k = 1

In [0]:
accuracy = predict_on_ROLE_88(albert_tokenizer, albert_model, df_ROLE88, 1)
print("ACCURACY = {}".format(accuracy))

[CLS] The librarian documented which journalist the celebrities had  [MASK] . [SEP] 
Predictions:
- interviewed
FAIL: predicted != expected
GOOD: predicted == target
expected: avoided|hated|read|met, target: interviewed


[CLS] The librarian documented which celebrities the journalist had  [MASK] . [SEP] 
Predictions:
- written
FAIL: predicted != expected
expected: interviewed, target: interviewed


[CLS] the old widower remembered which ghost the villager had  [MASK] . [SEP] 
Predictions:
- summoned
FAIL: predicted != expected
expected: seen, target: haunted


[CLS] the old widower remembered which villager the ghost had  [MASK] . [SEP] 
Predictions:
- attacked
FAIL: predicted != expected
expected: haunted, target: haunted


[CLS] the firefighter reported which paramedic the victim had  [MASK] . [SEP] 
Predictions:
- hired
FAIL: predicted != expected
expected: thanked, target: saved


[CLS] the firefighter reported which victim the paramedic had  [MASK] . [SEP] 
Predictions:
- identif

k = 3

In [0]:
accuracy = predict_on_ROLE_88(albert_tokenizer, albert_model, df_ROLE88, 3)
print("ACCURACY = {}".format(accuracy))

[CLS] The librarian documented which journalist the celebrities had  [MASK] . [SEP] 
Predictions:
- interviewed
- contacted
- hired
FAIL: predicted != expected
GOOD: predicted == target
expected: avoided|hated|read|met, target: interviewed


[CLS] The librarian documented which celebrities the journalist had  [MASK] . [SEP] 
Predictions:
- written
- done
- authored
FAIL: predicted != expected
expected: interviewed, target: interviewed


[CLS] the old widower remembered which ghost the villager had  [MASK] . [SEP] 
Predictions:
- summoned
- killed
- slain
FAIL: predicted != expected
expected: seen, target: haunted


[CLS] the old widower remembered which villager the ghost had  [MASK] . [SEP] 
Predictions:
- attacked
- murdered
- killed
FAIL: predicted != expected
expected: haunted, target: haunted


[CLS] the firefighter reported which paramedic the victim had  [MASK] . [SEP] 
Predictions:
- hired
- been
- contacted
FAIL: predicted != expected
expected: thanked, target: saved


[CLS] t

k = 5

In [0]:
accuracy = predict_on_ROLE_88(albert_tokenizer, albert_model, df_ROLE88, 5)
print("ACCURACY = {}".format(accuracy))

[CLS] The librarian documented which journalist the celebrities had  [MASK] . [SEP] 
Predictions:
- interviewed
- contacted
- hired
- consulted
- befriended
FAIL: predicted != expected
GOOD: predicted == target
expected: avoided|hated|read|met, target: interviewed


[CLS] The librarian documented which celebrities the journalist had  [MASK] . [SEP] 
Predictions:
- written
- done
- authored
- addressed
- interviewed
PASS: predicted == expected
GOOD: predicted == target
expected: interviewed, target: interviewed


[CLS] the old widower remembered which ghost the villager had  [MASK] . [SEP] 
Predictions:
- summoned
- killed
- slain
- murdered
- seen
PASS: predicted == expected
expected: seen, target: haunted


[CLS] the old widower remembered which villager the ghost had  [MASK] . [SEP] 
Predictions:
- attacked
- murdered
- killed
- visited
- summoned
FAIL: predicted != expected
expected: haunted, target: haunted


[CLS] the firefighter reported which paramedic the victim had  [MASK] . [

k = 10

In [0]:
accuracy = predict_on_ROLE_88(albert_tokenizer, albert_model, df_ROLE88, 10)
print("ACCURACY = {}".format(accuracy))

[CLS] The librarian documented which journalist the celebrities had  [MASK] . [SEP] 
Predictions:
- interviewed
- contacted
- hired
- consulted
- befriended
- been
- visited
- encountered
- recruited
- employed
FAIL: predicted != expected
GOOD: predicted == target
expected: avoided|hated|read|met, target: interviewed


[CLS] The librarian documented which celebrities the journalist had  [MASK] . [SEP] 
Predictions:
- written
- done
- authored
- addressed
- interviewed
- visited
- taken
- performed
- contacted
- encountered
PASS: predicted == expected
GOOD: predicted == target
expected: interviewed, target: interviewed


[CLS] the old widower remembered which ghost the villager had  [MASK] . [SEP] 
Predictions:
- summoned
- killed
- slain
- murdered
- seen
- encountered
- chased
- attacked
- found
- stabbed
PASS: predicted == expected
expected: seen, target: haunted


[CLS] the old widower remembered which villager the ghost had  [MASK] . [SEP] 
Predictions:
- attacked
- murdered
- kill

## Albert (albert-large-v2)


In [0]:
from transformers import AlbertTokenizer, TFAlbertForMaskedLM

weights = 'albert-large-v2'
albert_large_tokenizer = AlbertTokenizer.from_pretrained(weights)
albert_large_model = TFAlbertForMaskedLM.from_pretrained(weights)

k = 1

In [0]:
accuracy = predict_on_ROLE_88(albert_large_tokenizer, albert_large_model, df_ROLE88, 1)
print("ACCURACY = {}".format(accuracy))

[CLS] The librarian documented which journalist the celebrities had  [MASK] . [SEP] 
Predictions:
- interviewed
FAIL: predicted != expected
GOOD: predicted == target
expected: avoided|hated|read|met, target: interviewed


[CLS] The librarian documented which celebrities the journalist had  [MASK] . [SEP] 
Predictions:
- interviewed
PASS: predicted == expected
GOOD: predicted == target
expected: interviewed, target: interviewed


[CLS] the old widower remembered which ghost the villager had  [MASK] . [SEP] 
Predictions:
- seen
PASS: predicted == expected
expected: seen, target: haunted


[CLS] the old widower remembered which villager the ghost had  [MASK] . [SEP] 
Predictions:
- been
FAIL: predicted != expected
expected: haunted, target: haunted


[CLS] the firefighter reported which paramedic the victim had  [MASK] . [SEP] 
Predictions:
- hired
FAIL: predicted != expected
expected: thanked, target: saved


[CLS] the firefighter reported which victim the paramedic had  [MASK] . [SEP] 


k = 3

In [0]:
accuracy = predict_on_ROLE_88(albert_large_tokenizer, albert_large_model, df_ROLE88, 3)
print("ACCURACY = {}".format(accuracy))

[CLS] The librarian documented which journalist the celebrities had  [MASK] . [SEP] 
Predictions:
- interviewed
- hired
- contacted
FAIL: predicted != expected
GOOD: predicted == target
expected: avoided|hated|read|met, target: interviewed


[CLS] The librarian documented which celebrities the journalist had  [MASK] . [SEP] 
Predictions:
- interviewed
- photographed
- befriended
PASS: predicted == expected
GOOD: predicted == target
expected: interviewed, target: interviewed


[CLS] the old widower remembered which ghost the villager had  [MASK] . [SEP] 
Predictions:
- seen
- slain
- been
PASS: predicted == expected
expected: seen, target: haunted


[CLS] the old widower remembered which villager the ghost had  [MASK] . [SEP] 
Predictions:
- been
- slain
- eaten
FAIL: predicted != expected
expected: haunted, target: haunted


[CLS] the firefighter reported which paramedic the victim had  [MASK] . [SEP] 
Predictions:
- hired
- been
- contacted
FAIL: predicted != expected
expected: thanke

k = 5

In [0]:
accuracy = predict_on_ROLE_88(albert_large_tokenizer, albert_large_model, df_ROLE88, 5)
print("ACCURACY = {}".format(accuracy))

[CLS] The librarian documented which journalist the celebrities had  [MASK] . [SEP] 
Predictions:
- interviewed
- hired
- contacted
- consulted
- been
FAIL: predicted != expected
GOOD: predicted == target
expected: avoided|hated|read|met, target: interviewed


[CLS] The librarian documented which celebrities the journalist had  [MASK] . [SEP] 
Predictions:
- interviewed
- photographed
- befriended
- contacted
- met
PASS: predicted == expected
GOOD: predicted == target
expected: interviewed, target: interviewed


[CLS] the old widower remembered which ghost the villager had  [MASK] . [SEP] 
Predictions:
- seen
- slain
- been
- woken
- led
PASS: predicted == expected
expected: seen, target: haunted


[CLS] the old widower remembered which villager the ghost had  [MASK] . [SEP] 
Predictions:
- been
- slain
- eaten
- woken
- visited
FAIL: predicted != expected
expected: haunted, target: haunted


[CLS] the firefighter reported which paramedic the victim had  [MASK] . [SEP] 
Predictions:
- 

k = 10

In [0]:
accuracy = predict_on_ROLE_88(albert_large_tokenizer, albert_large_model, df_ROLE88, 10)
print("ACCURACY = {}".format(accuracy))

[CLS] The librarian documented which journalist the celebrities had  [MASK] . [SEP] 
Predictions:
- interviewed
- hired
- contacted
- consulted
- been
- befriended
- dated
- employed
- met
- chosen
PASS: predicted == expected
GOOD: predicted == target
expected: avoided|hated|read|met, target: interviewed


[CLS] The librarian documented which celebrities the journalist had  [MASK] . [SEP] 
Predictions:
- interviewed
- photographed
- befriended
- contacted
- met
- visited
- seen
- encountered
- filmed
- dated
PASS: predicted == expected
GOOD: predicted == target
expected: interviewed, target: interviewed


[CLS] the old widower remembered which ghost the villager had  [MASK] . [SEP] 
Predictions:
- seen
- slain
- been
- woken
- led
- awakened
- followed
- haunted
- fallen
- witnessed
PASS: predicted == expected
GOOD: predicted == target
expected: seen, target: haunted


[CLS] the old widower remembered which villager the ghost had  [MASK] . [SEP] 
Predictions:
- been
- slain
- eaten
- w

#  NEG-88-NAT

In [0]:
df_NEG88NAT = pd.read_csv(os.path.join("datasets","NEG-88-NAT.tsv"),sep='\t')
df_NEG88NAT.head()

,item,context_aff,context_neg,target_aff,target_neg,licensing
0,0,"With proper equipment, scuba-diving is very","With proper equipment, scuba-diving isn't very",safe,dangerous,Y
1,1,Traveling in Baghdad is very,Traveling in Baghdad isn't very,dangerous,safe,Y
2,2,"In moderation, drinking red wine is","In moderation, drinking red wine isn't",good,bad,Y
3,3,Drinking tap water in developing countries is ...,Drinking tap water in developing countries isn...,dangerous,safe,Y
4,4,Secondhand furniture is very,Secondhand furniture isn't very,cheap,expensive,Y


In [0]:
def add_special_tokens(context1, context2):
  context1 = context1 + MASK
  context1 = CLS + context1 + " ." + SEP
  context2 = context2 + MASK
  context2 = CLS + context2 + " ." + SEP
  return context1, context2

In [0]:
for i in range(0, df_NEG88NAT.shape[0]):
  df_NEG88NAT.at[i, "context_aff"], df_NEG88NAT.at[i, "context_neg"] = add_special_tokens(df_NEG88NAT.at[i, "context_aff"], df_NEG88NAT.at[i, "context_neg"])
  print("{} | {}".format(df_NEG88NAT.at[i, "context_aff"], df_NEG88NAT.at[i, "context_neg"]))

[CLS] With proper equipment, scuba-diving is very [MASK] . [SEP]  | [CLS] With proper equipment, scuba-diving isn't very [MASK] . [SEP] 
[CLS] Traveling in Baghdad is very [MASK] . [SEP]  | [CLS] Traveling in Baghdad isn't very [MASK] . [SEP] 
[CLS] In moderation, drinking red wine is [MASK] . [SEP]  | [CLS] In moderation, drinking red wine isn't [MASK] . [SEP] 
[CLS] Drinking tap water in developing countries is very [MASK] . [SEP]  | [CLS] Drinking tap water in developing countries isn't very [MASK] . [SEP] 
[CLS] Secondhand furniture is very [MASK] . [SEP]  | [CLS] Secondhand furniture isn't very [MASK] . [SEP] 
[CLS] Most smokers find that quitting is very [MASK] . [SEP]  | [CLS] Most smokers find that quitting isn't very [MASK] . [SEP] 
[CLS] Old computers may be [MASK] . [SEP]  | [CLS] Old computers may not be [MASK] . [SEP] 
[CLS] A fast food dinner on a first date is very [MASK] . [SEP]  | [CLS] A fast food dinner on a first date isn't very [MASK] . [SEP] 
[CLS] Bulletproof ves

In [0]:
def predict_word(context, tokenizer, model, k=1):
  tokenized_text = tokenizer.tokenize(context)
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

  segments_ids = [0] * len(tokenized_text)
  #print(tokenized_text)
  masked_index = tokenized_text.index('[MASK]') 

  predictions = model.call(np.array([indexed_tokens]))
  predicted_index = np.argmax(predictions[0][0,masked_index,:])
  predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
  res = np.argsort(predictions[0][0,masked_index,:])[::-1][:k]
  return res

In [0]:
def predict_on_NEG88NAT(tokenizer, model, df_NEG88NAT, k=1):
  count = 0
  passed_aff = 0
  failed_aff = 0

  passed_neg = 0
  failed_neg = 0
  for i in range(0, df_NEG88NAT.shape[0]):
    print("{}".format(df_NEG88NAT.iloc[i]["context_aff"]))
    print("{}".format(df_NEG88NAT.iloc[i]["context_neg"]))
    predicted_ids_aff = predict_word(df_NEG88NAT.iloc[i]["context_aff"], tokenizer, model, k)
    predicted_ids_neg = predict_word(df_NEG88NAT.iloc[i]["context_neg"], tokenizer, model, k)

    predicted_words_aff = [tokenizer.convert_ids_to_tokens([word])[0] for word in predicted_ids_aff]
    predicted_words_neg = [tokenizer.convert_ids_to_tokens([word])[0] for word in predicted_ids_neg]

    predicted_words_aff = [word.replace('▁', '') for word in predicted_words_aff]
    predicted_words_neg = [word.replace('▁', '') for word in predicted_words_neg]

    print('Predictions aff:')
    for pred in predicted_words_aff:
      print('- {}'.format(pred))

    print('Predictions neg:')
    for pred in predicted_words_neg:
      print('- {}'.format(pred))
    print('Predictions aff: {}'.format(predicted_words_aff))
    print('Predictions neg: {}'.format(predicted_words_neg))

    exp_aff = df_NEG88NAT.iloc[i]["target_aff"]
    exp_neg = df_NEG88NAT.iloc[i]["target_neg"]

    if check_predicted_words(predicted_words_aff, exp_aff):
      passed_aff += 1
      print("PASS AFF: predicted == expected")
    else:
      failed_aff += 1
      print("FAIL AFF: predicted != expected") 

    if check_predicted_words(predicted_words_neg, exp_neg):
      passed_neg += 1
      print("PASS NEG: predicted == expected")
    else:
      failed_neg += 1
      print("FAIL NEG: predicted != expected") 
    
    print("target_aff: {}, target_neg: {}".format(df_NEG88NAT.iloc[i]["target_aff"], 
                                                  df_NEG88NAT.iloc[i]["target_neg"]))
    count += 1
    print("\n")
  accuracy_aff = (float)(passed_aff / count) 
  accuracy_neg = (float)(passed_neg / count) 
  print("------------------------------------")
  print("POSITIVE  ALL: {} | PASSED: {} | FAILED: {}".format(count, passed_aff, failed_aff))
  print("NEGTIVE   ALL: {} | PASSED: {} | FAILED: {}".format(count, passed_neg, failed_neg))
  print("------------------------------------")
  return round(accuracy_aff, 2), round(accuracy_neg, 2)

## Bert

In [0]:
from transformers import BertTokenizer, TFBertForMaskedLM

weights = 'bert-base-uncased'
bert_tokenizer = BertTokenizer.from_pretrained(weights)
bert_model = TFBertForMaskedLM.from_pretrained(weights)

k = 1

In [0]:
accuracy_aff, accuracy_neg = predict_on_NEG88NAT(bert_tokenizer, bert_model, df_NEG88NAT, 1)
print("ACCURACY AFF = {}, ACCURACY NEG = {}".format(accuracy_aff, accuracy_neg))

[CLS] With proper equipment, scuba-diving is very [MASK] . [SEP] 
[CLS] With proper equipment, scuba-diving isn't very [MASK] . [SEP] 
Predictions aff:
- easy
Predictions neg:
- safe
Predictions aff: ['easy']
Predictions neg: ['safe']
FAIL AFF: predicted != expected
FAIL NEG: predicted != expected
target_aff: safe, target_neg: dangerous


[CLS] Traveling in Baghdad is very [MASK] . [SEP] 
[CLS] Traveling in Baghdad isn't very [MASK] . [SEP] 
Predictions aff:
- difficult
Predictions neg:
- easy
Predictions aff: ['difficult']
Predictions neg: ['easy']
FAIL AFF: predicted != expected
FAIL NEG: predicted != expected
target_aff: dangerous, target_neg: safe


[CLS] In moderation, drinking red wine is [MASK] . [SEP] 
[CLS] In moderation, drinking red wine isn't [MASK] . [SEP] 
Predictions aff:
- prohibited
Predictions neg:
- easy
Predictions aff: ['prohibited']
Predictions neg: ['easy']
FAIL AFF: predicted != expected
FAIL NEG: predicted != expected
target_aff: good, target_neg: bad


[CLS] D

k = 3

In [0]:
accuracy_aff, accuracy_neg = predict_on_NEG88NAT(bert_tokenizer, bert_model, df_NEG88NAT, 3)
print("ACCURACY AFF = {}, ACCURACY NEG = {}".format(accuracy_aff, accuracy_neg))

[CLS] With proper equipment, scuba-diving is very [MASK] . [SEP] 
[CLS] With proper equipment, scuba-diving isn't very [MASK] . [SEP] 
Predictions aff:
- easy
- difficult
- safe
Predictions neg:
- safe
- easy
- difficult
Predictions aff: ['easy', 'difficult', 'safe']
Predictions neg: ['safe', 'easy', 'difficult']
PASS AFF: predicted == expected
FAIL NEG: predicted != expected
target_aff: safe, target_neg: dangerous


[CLS] Traveling in Baghdad is very [MASK] . [SEP] 
[CLS] Traveling in Baghdad isn't very [MASK] . [SEP] 
Predictions aff:
- difficult
- hard
- expensive
Predictions neg:
- easy
- pleasant
- difficult
Predictions aff: ['difficult', 'hard', 'expensive']
Predictions neg: ['easy', 'pleasant', 'difficult']
FAIL AFF: predicted != expected
FAIL NEG: predicted != expected
target_aff: dangerous, target_neg: safe


[CLS] In moderation, drinking red wine is [MASK] . [SEP] 
[CLS] In moderation, drinking red wine isn't [MASK] . [SEP] 
Predictions aff:
- prohibited
- recommended
- forbi

k = 5

In [0]:
accuracy_aff, accuracy_neg = predict_on_NEG88NAT(bert_tokenizer, bert_model, df_NEG88NAT, 5)
print("ACCURACY AFF = {}, ACCURACY NEG = {}".format(accuracy_aff, accuracy_neg))

[CLS] With proper equipment, scuba-diving is very [MASK] . [SEP] 
[CLS] With proper equipment, scuba-diving isn't very [MASK] . [SEP] 
Predictions aff:
- easy
- difficult
- safe
- common
- popular
Predictions neg:
- safe
- easy
- difficult
- common
- dangerous
Predictions aff: ['easy', 'difficult', 'safe', 'common', 'popular']
Predictions neg: ['safe', 'easy', 'difficult', 'common', 'dangerous']
PASS AFF: predicted == expected
PASS NEG: predicted == expected
target_aff: safe, target_neg: dangerous


[CLS] Traveling in Baghdad is very [MASK] . [SEP] 
[CLS] Traveling in Baghdad isn't very [MASK] . [SEP] 
Predictions aff:
- difficult
- hard
- expensive
- easy
- dangerous
Predictions neg:
- easy
- pleasant
- difficult
- hard
- comfortable
Predictions aff: ['difficult', 'hard', 'expensive', 'easy', 'dangerous']
Predictions neg: ['easy', 'pleasant', 'difficult', 'hard', 'comfortable']
PASS AFF: predicted == expected
FAIL NEG: predicted != expected
target_aff: dangerous, target_neg: safe


[C

k = 10

In [0]:
accuracy_aff, accuracy_neg = predict_on_NEG88NAT(bert_tokenizer, bert_model, df_NEG88NAT, 10)
print("ACCURACY AFF = {}, ACCURACY NEG = {}".format(accuracy_aff, accuracy_neg))

[CLS] With proper equipment, scuba-diving is very [MASK] . [SEP] 
[CLS] With proper equipment, scuba-diving isn't very [MASK] . [SEP] 
Predictions aff:
- easy
- difficult
- safe
- common
- popular
- dangerous
- practical
- possible
- good
- challenging
Predictions neg:
- safe
- easy
- difficult
- common
- dangerous
- popular
- practical
- good
- hard
- uncommon
Predictions aff: ['easy', 'difficult', 'safe', 'common', 'popular', 'dangerous', 'practical', 'possible', 'good', 'challenging']
Predictions neg: ['safe', 'easy', 'difficult', 'common', 'dangerous', 'popular', 'practical', 'good', 'hard', 'uncommon']
PASS AFF: predicted == expected
PASS NEG: predicted == expected
target_aff: safe, target_neg: dangerous


[CLS] Traveling in Baghdad is very [MASK] . [SEP] 
[CLS] Traveling in Baghdad isn't very [MASK] . [SEP] 
Predictions aff:
- difficult
- hard
- expensive
- easy
- dangerous
- tough
- important
- common
- challenging
- uncomfortable
Predictions neg:
- easy
- pleasant
- difficult
-

## DistilBert (distilbert-base-cased)

In [0]:
from transformers import DistilBertTokenizer, TFDistilBertForMaskedLM

weights = 'distilbert-base-cased'
distilbert_tokenizer = DistilBertTokenizer.from_pretrained(weights)
distilbert_model = TFDistilBertForMaskedLM.from_pretrained(weights)

k = 1

In [0]:
accuracy_aff, accuracy_neg = predict_on_NEG88NAT(distilbert_tokenizer, distilbert_model, df_NEG88NAT, 1)
print("ACCURACY AFF = {}, ACCURACY NEG = {}".format(accuracy_aff, accuracy_neg))

[CLS] With proper equipment, scuba-diving is very [MASK] . [SEP] 
[CLS] With proper equipment, scuba-diving isn't very [MASK] . [SEP] 
Predictions aff:
- difficult
Predictions neg:
- dangerous
Predictions aff: ['difficult']
Predictions neg: ['dangerous']
FAIL AFF: predicted != expected
PASS NEG: predicted == expected
target_aff: safe, target_neg: dangerous


[CLS] Traveling in Baghdad is very [MASK] . [SEP] 
[CLS] Traveling in Baghdad isn't very [MASK] . [SEP] 
Predictions aff:
- difficult
Predictions neg:
- easy
Predictions aff: ['difficult']
Predictions neg: ['easy']
FAIL AFF: predicted != expected
FAIL NEG: predicted != expected
target_aff: dangerous, target_neg: safe


[CLS] In moderation, drinking red wine is [MASK] . [SEP] 
[CLS] In moderation, drinking red wine isn't [MASK] . [SEP] 
Predictions aff:
- prohibited
Predictions neg:
- permitted
Predictions aff: ['prohibited']
Predictions neg: ['permitted']
FAIL AFF: predicted != expected
FAIL NEG: predicted != expected
target_aff: g

k = 3

In [0]:
accuracy_aff, accuracy_neg = predict_on_NEG88NAT(distilbert_tokenizer, distilbert_model, df_NEG88NAT, 3)
print("ACCURACY AFF = {}, ACCURACY NEG = {}".format(accuracy_aff, accuracy_neg))

[CLS] With proper equipment, scuba-diving is very [MASK] . [SEP] 
[CLS] With proper equipment, scuba-diving isn't very [MASK] . [SEP] 
Predictions aff:
- difficult
- popular
- dangerous
Predictions neg:
- dangerous
- difficult
- effective
Predictions aff: ['difficult', 'popular', 'dangerous']
Predictions neg: ['dangerous', 'difficult', 'effective']
FAIL AFF: predicted != expected
PASS NEG: predicted == expected
target_aff: safe, target_neg: dangerous


[CLS] Traveling in Baghdad is very [MASK] . [SEP] 
[CLS] Traveling in Baghdad isn't very [MASK] . [SEP] 
Predictions aff:
- difficult
- common
- rare
Predictions neg:
- easy
- uncommon
- common
Predictions aff: ['difficult', 'common', 'rare']
Predictions neg: ['easy', 'uncommon', 'common']
FAIL AFF: predicted != expected
FAIL NEG: predicted != expected
target_aff: dangerous, target_neg: safe


[CLS] In moderation, drinking red wine is [MASK] . [SEP] 
[CLS] In moderation, drinking red wine isn't [MASK] . [SEP] 
Predictions aff:
- prohibit

k = 5

In [0]:
accuracy_aff, accuracy_neg = predict_on_NEG88NAT(distilbert_tokenizer, distilbert_model, df_NEG88NAT, 5)
print("ACCURACY AFF = {}, ACCURACY NEG = {}".format(accuracy_aff, accuracy_neg))

[CLS] With proper equipment, scuba-diving is very [MASK] . [SEP] 
[CLS] With proper equipment, scuba-diving isn't very [MASK] . [SEP] 
Predictions aff:
- difficult
- popular
- dangerous
- effective
- challenging
Predictions neg:
- dangerous
- difficult
- effective
- easy
- practical
Predictions aff: ['difficult', 'popular', 'dangerous', 'effective', 'challenging']
Predictions neg: ['dangerous', 'difficult', 'effective', 'easy', 'practical']
FAIL AFF: predicted != expected
PASS NEG: predicted == expected
target_aff: safe, target_neg: dangerous


[CLS] Traveling in Baghdad is very [MASK] . [SEP] 
[CLS] Traveling in Baghdad isn't very [MASK] . [SEP] 
Predictions aff:
- difficult
- common
- rare
- easy
- popular
Predictions neg:
- easy
- uncommon
- common
- helpful
- difficult
Predictions aff: ['difficult', 'common', 'rare', 'easy', 'popular']
Predictions neg: ['easy', 'uncommon', 'common', 'helpful', 'difficult']
FAIL AFF: predicted != expected
FAIL NEG: predicted != expected
target_aff: 

k = 10

In [0]:
accuracy_aff, accuracy_neg = predict_on_NEG88NAT(distilbert_tokenizer, distilbert_model, df_NEG88NAT, 10)
print("ACCURACY AFF = {}, ACCURACY NEG = {}".format(accuracy_aff, accuracy_neg))

[CLS] With proper equipment, scuba-diving is very [MASK] . [SEP] 
[CLS] With proper equipment, scuba-diving isn't very [MASK] . [SEP] 
Predictions aff:
- difficult
- popular
- dangerous
- effective
- challenging
- easy
- competitive
- important
- risky
- common
Predictions neg:
- dangerous
- difficult
- effective
- easy
- practical
- uncommon
- popular
- important
- useful
- risky
Predictions aff: ['difficult', 'popular', 'dangerous', 'effective', 'challenging', 'easy', 'competitive', 'important', 'risky', 'common']
Predictions neg: ['dangerous', 'difficult', 'effective', 'easy', 'practical', 'uncommon', 'popular', 'important', 'useful', 'risky']
FAIL AFF: predicted != expected
PASS NEG: predicted == expected
target_aff: safe, target_neg: dangerous


[CLS] Traveling in Baghdad is very [MASK] . [SEP] 
[CLS] Traveling in Baghdad isn't very [MASK] . [SEP] 
Predictions aff:
- difficult
- common
- rare
- easy
- popular
- expensive
- challenging
- important
- slow
- uncommon
Predictions neg:

## Albert (albert-base-v2) 

In [0]:
from transformers import AlbertTokenizer, TFAlbertForMaskedLM

weights = 'albert-base-v2'
albert_tokenizer = AlbertTokenizer.from_pretrained(weights)
albert_model = TFAlbertForMaskedLM.from_pretrained(weights)

k = 1

In [0]:
accuracy_aff, accuracy_neg = predict_on_NEG88NAT(albert_tokenizer, albert_model, df_NEG88NAT, 1)
print("ACCURACY AFF = {}, ACCURACY NEG = {}".format(accuracy_aff, accuracy_neg))

[CLS] With proper equipment, scuba-diving is very [MASK] . [SEP] 
[CLS] With proper equipment, scuba-diving isn't very [MASK] . [SEP] 
Predictions aff:
- easy
Predictions neg:
- difficult
Predictions aff: ['easy']
Predictions neg: ['difficult']
FAIL AFF: predicted != expected
FAIL NEG: predicted != expected
target_aff: safe, target_neg: dangerous


[CLS] Traveling in Baghdad is very [MASK] . [SEP] 
[CLS] Traveling in Baghdad isn't very [MASK] . [SEP] 
Predictions aff:
- difficult
Predictions neg:
- easy
Predictions aff: ['difficult']
Predictions neg: ['easy']
FAIL AFF: predicted != expected
FAIL NEG: predicted != expected
target_aff: dangerous, target_neg: safe


[CLS] In moderation, drinking red wine is [MASK] . [SEP] 
[CLS] In moderation, drinking red wine isn't [MASK] . [SEP] 
Predictions aff:
- addictive
Predictions neg:
- uncommon
Predictions aff: ['addictive']
Predictions neg: ['uncommon']
FAIL AFF: predicted != expected
FAIL NEG: predicted != expected
target_aff: good, target_ne

k = 3

In [0]:
accuracy_aff, accuracy_neg = predict_on_NEG88NAT(albert_tokenizer, albert_model, df_NEG88NAT, 3)
print("ACCURACY AFF = {}, ACCURACY NEG = {}".format(accuracy_aff, accuracy_neg))

[CLS] With proper equipment, scuba-diving is very [MASK] . [SEP] 
[CLS] With proper equipment, scuba-diving isn't very [MASK] . [SEP] 
Predictions aff:
- easy
- simple
- straightforward
Predictions neg:
- difficult
- complicated
- risky
Predictions aff: ['easy', 'simple', 'straightforward']
Predictions neg: ['difficult', 'complicated', 'risky']
FAIL AFF: predicted != expected
FAIL NEG: predicted != expected
target_aff: safe, target_neg: dangerous


[CLS] Traveling in Baghdad is very [MASK] . [SEP] 
[CLS] Traveling in Baghdad isn't very [MASK] . [SEP] 
Predictions aff:
- difficult
- risky
- dangerous
Predictions neg:
- easy
- exciting
- convenient
Predictions aff: ['difficult', 'risky', 'dangerous']
Predictions neg: ['easy', 'exciting', 'convenient']
PASS AFF: predicted == expected
FAIL NEG: predicted != expected
target_aff: dangerous, target_neg: safe


[CLS] In moderation, drinking red wine is [MASK] . [SEP] 
[CLS] In moderation, drinking red wine isn't [MASK] . [SEP] 
Predictions aff

k = 5

In [0]:
accuracy_aff, accuracy_neg = predict_on_NEG88NAT(albert_tokenizer, albert_model, df_NEG88NAT, 5)
print("ACCURACY AFF = {}, ACCURACY NEG = {}".format(accuracy_aff, accuracy_neg))

[CLS] With proper equipment, scuba-diving is very [MASK] . [SEP] 
[CLS] With proper equipment, scuba-diving isn't very [MASK] . [SEP] 
Predictions aff:
- easy
- simple
- straightforward
- inexpensive
- risky
Predictions neg:
- difficult
- complicated
- risky
- expensive
- hazardous
Predictions aff: ['easy', 'simple', 'straightforward', 'inexpensive', 'risky']
Predictions neg: ['difficult', 'complicated', 'risky', 'expensive', 'hazardous']
FAIL AFF: predicted != expected
FAIL NEG: predicted != expected
target_aff: safe, target_neg: dangerous


[CLS] Traveling in Baghdad is very [MASK] . [SEP] 
[CLS] Traveling in Baghdad isn't very [MASK] . [SEP] 
Predictions aff:
- difficult
- risky
- dangerous
- expensive
- exciting
Predictions neg:
- easy
- exciting
- convenient
- enjoyable
- pleasant
Predictions aff: ['difficult', 'risky', 'dangerous', 'expensive', 'exciting']
Predictions neg: ['easy', 'exciting', 'convenient', 'enjoyable', 'pleasant']
PASS AFF: predicted == expected
FAIL NEG: predic

k = 10

In [0]:
accuracy_aff, accuracy_neg = predict_on_NEG88NAT(albert_tokenizer, albert_model, df_NEG88NAT, 10)
print("ACCURACY AFF = {}, ACCURACY NEG = {}".format(accuracy_aff, accuracy_neg))

[CLS] With proper equipment, scuba-diving is very [MASK] . [SEP] 
[CLS] With proper equipment, scuba-diving isn't very [MASK] . [SEP] 
Predictions aff:
- easy
- simple
- straightforward
- inexpensive
- risky
- economical
- lucrative
- enjoyable
- profitable
- exciting
Predictions neg:
- difficult
- complicated
- risky
- expensive
- hazardous
- dangerous
- scary
- bad
- painful
- noisy
Predictions aff: ['easy', 'simple', 'straightforward', 'inexpensive', 'risky', 'economical', 'lucrative', 'enjoyable', 'profitable', 'exciting']
Predictions neg: ['difficult', 'complicated', 'risky', 'expensive', 'hazardous', 'dangerous', 'scary', 'bad', 'painful', 'noisy']
FAIL AFF: predicted != expected
PASS NEG: predicted == expected
target_aff: safe, target_neg: dangerous


[CLS] Traveling in Baghdad is very [MASK] . [SEP] 
[CLS] Traveling in Baghdad isn't very [MASK] . [SEP] 
Predictions aff:
- difficult
- risky
- dangerous
- expensive
- exciting
- scary
- complicated
- easy
- tempting
- confusing
Pr

## Albert (albert-large-v2)

In [0]:
from transformers import AlbertTokenizer, TFAlbertForMaskedLM

weights = 'albert-large-v2'
albert_large_tokenizer = AlbertTokenizer.from_pretrained(weights)
albert_large_model = TFAlbertForMaskedLM.from_pretrained(weights)

K = 1

In [0]:
accuracy_aff, accuracy_neg = predict_on_NEG88NAT(albert_large_tokenizer, albert_large_model, df_NEG88NAT, 1)
print("ACCURACY AFF = {}, ACCURACY NEG = {}".format(accuracy_aff, accuracy_neg))

[CLS] With proper equipment, scuba-diving is very [MASK] . [SEP] 
[CLS] With proper equipment, scuba-diving isn't very [MASK] . [SEP] 
Predictions aff:
- easy
Predictions neg:
- difficult
Predictions aff: ['easy']
Predictions neg: ['difficult']
FAIL AFF: predicted != expected
FAIL NEG: predicted != expected
target_aff: safe, target_neg: dangerous


[CLS] Traveling in Baghdad is very [MASK] . [SEP] 
[CLS] Traveling in Baghdad isn't very [MASK] . [SEP] 
Predictions aff:
- difficult
Predictions neg:
- easy
Predictions aff: ['difficult']
Predictions neg: ['easy']
FAIL AFF: predicted != expected
FAIL NEG: predicted != expected
target_aff: dangerous, target_neg: safe


[CLS] In moderation, drinking red wine is [MASK] . [SEP] 
[CLS] In moderation, drinking red wine isn't [MASK] . [SEP] 
Predictions aff:
- acceptable
Predictions neg:
- acceptable
Predictions aff: ['acceptable']
Predictions neg: ['acceptable']
FAIL AFF: predicted != expected
FAIL NEG: predicted != expected
target_aff: good, tar

k = 3

In [0]:
accuracy_aff, accuracy_neg = predict_on_NEG88NAT(albert_large_tokenizer, albert_large_model, df_NEG88NAT, 3)
print("ACCURACY AFF = {}, ACCURACY NEG = {}".format(accuracy_aff, accuracy_neg))

[CLS] With proper equipment, scuba-diving is very [MASK] . [SEP] 
[CLS] With proper equipment, scuba-diving isn't very [MASK] . [SEP] 
Predictions aff:
- easy
- effective
- difficult
Predictions neg:
- difficult
- hard
- expensive
Predictions aff: ['easy', 'effective', 'difficult']
Predictions neg: ['difficult', 'hard', 'expensive']
FAIL AFF: predicted != expected
FAIL NEG: predicted != expected
target_aff: safe, target_neg: dangerous


[CLS] Traveling in Baghdad is very [MASK] . [SEP] 
[CLS] Traveling in Baghdad isn't very [MASK] . [SEP] 
Predictions aff:
- difficult
- expensive
- easy
Predictions neg:
- easy
- fun
- expensive
Predictions aff: ['difficult', 'expensive', 'easy']
Predictions neg: ['easy', 'fun', 'expensive']
FAIL AFF: predicted != expected
FAIL NEG: predicted != expected
target_aff: dangerous, target_neg: safe


[CLS] In moderation, drinking red wine is [MASK] . [SEP] 
[CLS] In moderation, drinking red wine isn't [MASK] . [SEP] 
Predictions aff:
- acceptable
- fine
- re

k = 5

In [0]:
accuracy_aff, accuracy_neg = predict_on_NEG88NAT(albert_large_tokenizer, albert_large_model, df_NEG88NAT, 5)
print("ACCURACY AFF = {}, ACCURACY NEG = {}".format(accuracy_aff, accuracy_neg))

[CLS] With proper equipment, scuba-diving is very [MASK] . [SEP] 
[CLS] With proper equipment, scuba-diving isn't very [MASK] . [SEP] 
Predictions aff:
- easy
- effective
- difficult
- possible
- enjoyable
Predictions neg:
- difficult
- hard
- expensive
- bad
- challenging
Predictions aff: ['easy', 'effective', 'difficult', 'possible', 'enjoyable']
Predictions neg: ['difficult', 'hard', 'expensive', 'bad', 'challenging']
FAIL AFF: predicted != expected
FAIL NEG: predicted != expected
target_aff: safe, target_neg: dangerous


[CLS] Traveling in Baghdad is very [MASK] . [SEP] 
[CLS] Traveling in Baghdad isn't very [MASK] . [SEP] 
Predictions aff:
- difficult
- expensive
- easy
- fast
- convenient
Predictions neg:
- easy
- fun
- expensive
- pleasant
- exciting
Predictions aff: ['difficult', 'expensive', 'easy', 'fast', 'convenient']
Predictions neg: ['easy', 'fun', 'expensive', 'pleasant', 'exciting']
FAIL AFF: predicted != expected
FAIL NEG: predicted != expected
target_aff: dangerous, t

k = 10

In [0]:
accuracy_aff, accuracy_neg = predict_on_NEG88NAT(albert_large_tokenizer, albert_large_model, df_NEG88NAT, 10)
print("ACCURACY AFF = {}, ACCURACY NEG = {}".format(accuracy_aff, accuracy_neg))

[CLS] With proper equipment, scuba-diving is very [MASK] . [SEP] 
[CLS] With proper equipment, scuba-diving isn't very [MASK] . [SEP] 
Predictions aff:
- easy
- effective
- difficult
- possible
- enjoyable
- simple
- exciting
- profitable
- straightforward
- lucrative
Predictions neg:
- difficult
- hard
- expensive
- bad
- challenging
- risky
- scary
- intimidating
- dangerous
- unusual
Predictions aff: ['easy', 'effective', 'difficult', 'possible', 'enjoyable', 'simple', 'exciting', 'profitable', 'straightforward', 'lucrative']
Predictions neg: ['difficult', 'hard', 'expensive', 'bad', 'challenging', 'risky', 'scary', 'intimidating', 'dangerous', 'unusual']
FAIL AFF: predicted != expected
PASS NEG: predicted == expected
target_aff: safe, target_neg: dangerous


[CLS] Traveling in Baghdad is very [MASK] . [SEP] 
[CLS] Traveling in Baghdad isn't very [MASK] . [SEP] 
Predictions aff:
- difficult
- expensive
- easy
- fast
- convenient
- economical
- pleasant
- crowded
- cheap
- busy
Predi

# RuBert

In [0]:
from transformers import AutoTokenizer, AutoModelWithLMHead

weights = "DrMatters/rubert_cased"
rubert_tokenizer = AutoTokenizer.from_pretrained(weights)
rubert_model = AutoModelWithLMHead.from_pretrained(weights)

# RUS-CPRAG-50

In [0]:
df_rus = pd.read_excel(os.path.join("datasets","RUS-CPRAG-50.xlsx"),  encoding="cp866")
df_rus.head()

,context1,context2,expected
0,В доме раздался громкий звонок.,Хозяйка пошла открыть,дверь
1,Животные с длинной шеей вольяжно ходили по вол...,Зоопарк славился своими,жирафами
2,Гости должны прийти с минуты на минуту.,Необходимо было накрыть на,стол
3,Десять лет назад его укусила за ногу злая собака.,"Это было давно, но на коже остался",шрам
4,Она купила билеты и стала ждать на железнодоро...,Скоро должен был приехать ее,поезд


In [0]:
def add_special_tokens(context1, context2):
  context2 = context2 + MASK
  context1 = CLS + context1 + SEP
  context2 = CLS + context2 + " ." + SEP
  return context1, context2

In [0]:
for i in range(0, df_rus.shape[0]):
  df_rus.at[i, "context1"], df_rus.at[i, "context2"] = add_special_tokens(df_rus.iloc[i]["context1"], df_rus.iloc[i]["context2"])
  print("{} | {}".format(df_rus.iloc[i]["context1"], df_rus.iloc[i]["context2"]))

[CLS] В доме раздался громкий звонок.   [SEP]  | [CLS] Хозяйка пошла открыть [MASK] . [SEP] 
[CLS] Животные с длинной шеей вольяжно ходили по вольеру. [SEP]  | [CLS] Зоопарк славился своими [MASK] . [SEP] 
[CLS] Гости должны прийти с минуты на минуту. [SEP]  | [CLS] Необходимо было накрыть на [MASK] . [SEP] 
[CLS] Десять лет назад его укусила за ногу злая собака. [SEP]  | [CLS] Это было давно, но на коже остался  [MASK] . [SEP] 
[CLS] Она купила билеты и стала ждать на железнодорожном вокзале. [SEP]  | [CLS] Скоро должен был приехать ее [MASK] . [SEP] 
[CLS] Студент повторяет пройденный материал и учит билеты. [SEP]  | [CLS] Завтра у него будет сложный [MASK] . [SEP] 
[CLS] Мальчик очень любит Масленницу. [SEP]  | [CLS] В этот празник его мама печет [MASK] . [SEP] 
[CLS] Путешественники решили разбить лагерь. [SEP]  | [CLS] Нужно было поставить палатку и развести [MASK] . [SEP] 
[CLS] На улице было холодно и ее руки замерзли. [SEP]  | [CLS] Чтобы согреть их, она достала [MASK] . [SEP] 

In [0]:
def predict_word(context1, context2, tokenizer, model, k=1):
  full_context = context1 + context2
  tokenized_text = tokenizer.tokenize(full_context)
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

  segments_ids = [0] * len(tokenized_text)
  masked_index = tokenized_text.index('[MASK]') 

  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])
  model.eval()
  with torch.no_grad():
    predictions = model(tokens_tensor, segments_tensors)
  predicted_index = np.argmax(predictions[0][0,masked_index,:])
  predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
  res = np.argsort(predictions[0][0, masked_index].numpy())[::-1][:k]
  return res

In [0]:
def check_predicted_words(predicted_words, expected_word):
  for predicted_word in predicted_words:
    if predicted_word == expected_word:
      return True
  return False 

In [0]:
def predict_on_rus(tokenizer, model, df_rus, k=1):
  count = 0
  passed = 0
  failed = 0
  for i in range(0, df_rus.shape[0]):
    print("{} | {}".format(df_rus.iloc[i]["context1"], df_rus.iloc[i]["context2"]))
    predicted_ids = predict_word(df_rus.iloc[i]["context1"], df_rus.iloc[i]["context2"], tokenizer, model, k)
    predicted_words = [tokenizer.convert_ids_to_tokens([word])[0] for word in predicted_ids]
    predicted_words = [word.replace('▁', '') for word in predicted_words]
    #print('Predictions: {}'.format(predicted_words))
    print('Predictions:')
    for pred in predicted_words:
      print('- {}'.format(pred))
    if check_predicted_words(predicted_words, df_rus.iloc[i]["expected"]):
      passed += 1
      print("PASS: predicted == expected")
    else:
      failed += 1
      print("FAIL: predicted != expected") 
    count += 1
    print("\n")
  accuracy = (float)(passed / count) 
  print("------------------------------------")
  print("ALL: {} | PASSED: {} | FAILED: {}".format(count, passed, failed))
  print("------------------------------------")
  return round(accuracy, 2)

k = 1

In [0]:
accuracy = predict_on_rus(rubert_tokenizer, rubert_model, df_rus, 1)
print("ACCURACY = {}".format(accuracy))

[CLS] В доме раздался громкий звонок.   [SEP]  | [CLS] Хозяйка пошла открыть [MASK] . [SEP] 
Predictions:
- дверь
PASS: predicted == expected


[CLS] Животные с длинной шеей вольяжно ходили по вольеру. [SEP]  | [CLS] Зоопарк славился своими [MASK] . [SEP] 
Predictions:
- особенностями
FAIL: predicted != expected


[CLS] Гости должны прийти с минуты на минуту. [SEP]  | [CLS] Необходимо было накрыть на [MASK] . [SEP] 
Predictions:
- стол
PASS: predicted == expected


[CLS] Десять лет назад его укусила за ногу злая собака. [SEP]  | [CLS] Это было давно, но на коже остался  [MASK] . [SEP] 
Predictions:
- шрам
PASS: predicted == expected


[CLS] Она купила билеты и стала ждать на железнодорожном вокзале. [SEP]  | [CLS] Скоро должен был приехать ее [MASK] . [SEP] 
Predictions:
- муж
FAIL: predicted != expected


[CLS] Студент повторяет пройденный материал и учит билеты. [SEP]  | [CLS] Завтра у него будет сложный [MASK] . [SEP] 
Predictions:
- билеты
FAIL: predicted != expected


[CLS] Мальчи

k = 3

In [0]:
accuracy = predict_on_rus(rubert_tokenizer, rubert_model, df_rus, 3)
print("ACCURACY = {}".format(accuracy))

[CLS] В доме раздался громкий звонок.   [SEP]  | [CLS] Хозяйка пошла открыть [MASK] . [SEP] 
Predictions:
- дверь
- двери
- огонь
PASS: predicted == expected


[CLS] Животные с длинной шеей вольяжно ходили по вольеру. [SEP]  | [CLS] Зоопарк славился своими [MASK] . [SEP] 
Predictions:
- особенностями
- собаками
- размерами
FAIL: predicted != expected


[CLS] Гости должны прийти с минуты на минуту. [SEP]  | [CLS] Необходимо было накрыть на [MASK] . [SEP] 
Predictions:
- стол
- пол
- землю
PASS: predicted == expected


[CLS] Десять лет назад его укусила за ногу злая собака. [SEP]  | [CLS] Это было давно, но на коже остался  [MASK] . [SEP] 
Predictions:
- шрам
- след
- отпечаток
PASS: predicted == expected


[CLS] Она купила билеты и стала ждать на железнодорожном вокзале. [SEP]  | [CLS] Скоро должен был приехать ее [MASK] . [SEP] 
Predictions:
- муж
- отец
- супруг
FAIL: predicted != expected


[CLS] Студент повторяет пройденный материал и учит билеты. [SEP]  | [CLS] Завтра у него будет 

k = 5

In [0]:
accuracy = predict_on_rus(rubert_tokenizer, rubert_model, df_rus, 5)
print("ACCURACY = {}".format(accuracy))

[CLS] В доме раздался громкий звонок.   [SEP]  | [CLS] Хозяйка пошла открыть [MASK] . [SEP] 
Predictions:
- дверь
- двери
- огонь
- окно
- сейф
PASS: predicted == expected


[CLS] Животные с длинной шеей вольяжно ходили по вольеру. [SEP]  | [CLS] Зоопарк славился своими [MASK] . [SEP] 
Predictions:
- особенностями
- собаками
- размерами
- традициями
- способностями
FAIL: predicted != expected


[CLS] Гости должны прийти с минуты на минуту. [SEP]  | [CLS] Необходимо было накрыть на [MASK] . [SEP] 
Predictions:
- стол
- пол
- землю
- полу
- солнце
PASS: predicted == expected


[CLS] Десять лет назад его укусила за ногу злая собака. [SEP]  | [CLS] Это было давно, но на коже остался  [MASK] . [SEP] 
Predictions:
- шрам
- след
- отпечаток
- снег
- рисунок
PASS: predicted == expected


[CLS] Она купила билеты и стала ждать на железнодорожном вокзале. [SEP]  | [CLS] Скоро должен был приехать ее [MASK] . [SEP] 
Predictions:
- муж
- отец
- супруг
- сын
- жених
FAIL: predicted != expected


[CLS

k = 10

In [0]:
accuracy = predict_on_rus(rubert_tokenizer, rubert_model, df_rus, 10)
print("ACCURACY = {}".format(accuracy))

[CLS] В доме раздался громкий звонок.   [SEP]  | [CLS] Хозяйка пошла открыть [MASK] . [SEP] 
Predictions:
- дверь
- двери
- огонь
- окно
- сейф
- ##ся
- окна
- ворота
- его
- магазин
PASS: predicted == expected


[CLS] Животные с длинной шеей вольяжно ходили по вольеру. [SEP]  | [CLS] Зоопарк славился своими [MASK] . [SEP] 
Predictions:
- особенностями
- собаками
- размерами
- традициями
- способностями
- животными
- выступлениями
- условиями
- скульптурами
- номерами
FAIL: predicted != expected


[CLS] Гости должны прийти с минуты на минуту. [SEP]  | [CLS] Необходимо было накрыть на [MASK] . [SEP] 
Predictions:
- стол
- пол
- землю
- полу
- солнце
- льду
- месте
- доске
- столе
- дно
PASS: predicted == expected


[CLS] Десять лет назад его укусила за ногу злая собака. [SEP]  | [CLS] Это было давно, но на коже остался  [MASK] . [SEP] 
Predictions:
- шрам
- след
- отпечаток
- снег
- рисунок
- песок
- пепел
- зуб
- карандаш
- осадок
PASS: predicted == expected


[CLS] Она купила билеты и

# RUS-NEG-30-NAT

In [0]:
df_rus_neg = pd.read_excel(os.path.join("datasets", "RUS-NEG-30-NAT.xlsx"),  encoding="cp866")
df_rus_neg.head()

,context_aff,context_neg,expected_aff,expected_neg
0,В Якутии зимы очень * .,В Якутии зимы не очень * .,холодные,теплые
1,Слон - * животное.,Слон - не * животное.,большое,маленькое
2,Колибри - самые * птицы.,Колибри - не самые * птицы.,маленькие,большие
3,Учёные - * люди.,Учёные - не * люди.,умные,глупые
4,Шерсть котенка - * .,Шерсть котенка - не * .,мягкая,жесткая


In [0]:
def add_special_tokens(context1, context2):
  context1 = context1.replace("*", MASK)
  context1 = CLS + context1 + SEP
  context2 = context2.replace("*", MASK)
  context2 = CLS + context2 + SEP
  return context1, context2

In [0]:
for i in range(0, df_rus_neg.shape[0]):
  df_rus_neg.at[i, "context_aff"], df_rus_neg.at[i, "context_neg"] = add_special_tokens(df_rus_neg.at[i, "context_aff"], df_rus_neg.at[i, "context_neg"])
  print("{} | {}".format(df_rus_neg.at[i, "context_aff"], df_rus_neg.at[i, "context_neg"]))

[CLS] В Якутии зимы очень  [MASK] . [SEP]  | [CLS] В Якутии зимы не очень  [MASK] . [SEP] 
[CLS] Слон -  [MASK] животное. [SEP]  | [CLS] Слон - не  [MASK] животное. [SEP] 
[CLS] Колибри - самые  [MASK] птицы. [SEP]  | [CLS] Колибри - не самые  [MASK] птицы. [SEP] 
[CLS] Учёные -  [MASK] люди. [SEP]  | [CLS] Учёные - не  [MASK] люди. [SEP] 
[CLS] Шерсть котенка -  [MASK] . [SEP]  | [CLS] Шерсть котенка - не  [MASK] . [SEP] 
[CLS] Играть с огнём  [MASK] . [SEP]  | [CLS] Играть с огнём не  [MASK] . [SEP] 
[CLS] Пуховая перина на ощупь была  [MASK] . [SEP]  | [CLS] Пуховая перина на ощупь не была  [MASK] . [SEP] 
[CLS] Заточеный нож был  [MASK] . [SEP]  | [CLS] Заточенный нож не был  [MASK] . [SEP] 
[CLS] Карамельная конфета была  [MASK] . [SEP]  | [CLS] Карамельная конфета не была  [MASK] . [SEP] 
[CLS] Зарядка по утрам и здоровое питание -  [MASK] привычка. [SEP]  | [CLS] Зарядка по утрам и здоровое питание - не  [MASK] привычка. [SEP] 
[CLS] Штангист поднимает  [MASK] грузы. [SEP]  | [C

In [0]:
def predict_word(context, tokenizer, model, k=1):
  tokenized_text = tokenizer.tokenize(context)
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

  segments_ids = [0] * len(tokenized_text)
  #print(tokenized_text)
  masked_index = tokenized_text.index('[MASK]') 

  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])
  model.eval()
  with torch.no_grad():
    predictions = model(tokens_tensor, segments_tensors)
  predicted_index = np.argmax(predictions[0][0,masked_index,:])
  predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
  res = np.argsort(predictions[0][0, masked_index].numpy())[::-1][:k]
  return res

In [0]:
def predict_on_rus_neg(tokenizer, model, df_rus_neg, k=1):
  count = 0
  passed_aff = 0
  failed_aff = 0

  passed_neg = 0
  failed_neg = 0
  for i in range(0, df_rus_neg.shape[0]):
    print("{}".format(df_rus_neg.iloc[i]["context_aff"]))
    print("{}".format(df_rus_neg.iloc[i]["context_neg"]))
    predicted_ids_aff = predict_word(df_rus_neg.iloc[i]["context_aff"], tokenizer, model, k)
    predicted_ids_neg = predict_word(df_rus_neg.iloc[i]["context_neg"], tokenizer, model, k)

    predicted_words_aff = [tokenizer.convert_ids_to_tokens([word])[0] for word in predicted_ids_aff]
    predicted_words_neg = [tokenizer.convert_ids_to_tokens([word])[0] for word in predicted_ids_neg]

    predicted_words_aff = [word.replace('▁', '') for word in predicted_words_aff]
    predicted_words_neg = [word.replace('▁', '') for word in predicted_words_neg]

    print('Predictions aff:')
    for pred in predicted_words_aff:
      print('- {}'.format(pred))

    print('Predictions neg:')
    for pred in predicted_words_neg:
      print('- {}'.format(pred))
    print('Predictions aff: {}'.format(predicted_words_aff))
    print('Predictions neg: {}'.format(predicted_words_neg))

    exp_aff = df_rus_neg.iloc[i]["expected_aff"]
    exp_neg = df_rus_neg.iloc[i]["expected_neg"]

    if check_predicted_words(predicted_words_aff, exp_aff):
      passed_aff += 1
      print("PASS AFF: predicted == expected")
    else:
      failed_aff += 1
      print("FAIL AFF: predicted != expected") 

    if check_predicted_words(predicted_words_neg, exp_neg):
      passed_neg += 1
      print("PASS NEG: predicted == expected")
    else:
      failed_neg += 1
      print("FAIL NEG: predicted != expected") 
    
    print("target_aff: {}, target_neg: {}".format(df_rus_neg.iloc[i]["expected_aff"], 
                                                  df_rus_neg.iloc[i]["expected_neg"]))
    count += 1
    print("\n")
  accuracy_aff = (float)(passed_aff / count) 
  accuracy_neg = (float)(passed_neg / count) 
  print("------------------------------------")
  print("POSITIVE  ALL: {} | PASSED: {} | FAILED: {}".format(count, passed_aff, failed_aff))
  print("NEGTIVE   ALL: {} | PASSED: {} | FAILED: {}".format(count, passed_neg, failed_neg))
  print("------------------------------------")
  return round(accuracy_aff, 2), round(accuracy_neg, 2)

In [0]:
accuracy = predict_on_rus_neg(rubert_tokenizer, rubert_model, df_rus_neg, 1)
print("ACCURACY = {}".format(accuracy))

[CLS] В Якутии зимы очень  [MASK] . [SEP] 
[CLS] В Якутии зимы не очень  [MASK] . [SEP] 
Predictions aff:
- холодные
Predictions neg:
- холодные
Predictions aff: ['холодные']
Predictions neg: ['холодные']
PASS AFF: predicted == expected
FAIL NEG: predicted != expected
target_aff: холодные, target_neg: теплые


[CLS] Слон -  [MASK] животное. [SEP] 
[CLS] Слон - не  [MASK] животное. [SEP] 
Predictions aff:
- это
Predictions neg:
- одно
Predictions aff: ['это']
Predictions neg: ['одно']
FAIL AFF: predicted != expected
FAIL NEG: predicted != expected
target_aff: большое, target_neg: маленькое


[CLS] Колибри - самые  [MASK] птицы. [SEP] 
[CLS] Колибри - не самые  [MASK] птицы. [SEP] 
Predictions aff:
- крупные
Predictions neg:
- крупные
Predictions aff: ['крупные']
Predictions neg: ['крупные']
FAIL AFF: predicted != expected
FAIL NEG: predicted != expected
target_aff: маленькие, target_neg: большие


[CLS] Учёные -  [MASK] люди. [SEP] 
[CLS] Учёные - не  [MASK] люди. [SEP] 
Predictions aff

In [0]:
accuracy = predict_on_rus_neg(rubert_tokenizer, rubert_model, df_rus_neg, 3)
print("ACCURACY = {}".format(accuracy))

[CLS] В Якутии зимы очень  [MASK] . [SEP] 
[CLS] В Якутии зимы не очень  [MASK] . [SEP] 
Predictions aff:
- холодные
- суровые
- мягкие
Predictions neg:
- холодные
- суровые
- теплые
Predictions aff: ['холодные', 'суровые', 'мягкие']
Predictions neg: ['холодные', 'суровые', 'теплые']
PASS AFF: predicted == expected
PASS NEG: predicted == expected
target_aff: холодные, target_neg: теплые


[CLS] Слон -  [MASK] животное. [SEP] 
[CLS] Слон - не  [MASK] животное. [SEP] 
Predictions aff:
- это
- многозначное
- ночное
Predictions neg:
- одно
- домашнее
- просто
Predictions aff: ['это', 'многозначное', 'ночное']
Predictions neg: ['одно', 'домашнее', 'просто']
FAIL AFF: predicted != expected
FAIL NEG: predicted != expected
target_aff: большое, target_neg: маленькое


[CLS] Колибри - самые  [MASK] птицы. [SEP] 
[CLS] Колибри - не самые  [MASK] птицы. [SEP] 
Predictions aff:
- крупные
- маленькие
- большие
Predictions neg:
- крупные
- большие
- популярные
Predictions aff: ['крупные', 'маленькие'

In [0]:
accuracy = predict_on_rus_neg(rubert_tokenizer, rubert_model, df_rus_neg, 5)
print("ACCURACY = {}".format(accuracy))

[CLS] В Якутии зимы очень  [MASK] . [SEP] 
[CLS] В Якутии зимы не очень  [MASK] . [SEP] 
Predictions aff:
- холодные
- суровые
- мягкие
- теплые
- тёплые
Predictions neg:
- холодные
- суровые
- теплые
- тепло
- мягкие
Predictions aff: ['холодные', 'суровые', 'мягкие', 'теплые', 'тёплые']
Predictions neg: ['холодные', 'суровые', 'теплые', 'тепло', 'мягкие']
PASS AFF: predicted == expected
PASS NEG: predicted == expected
target_aff: холодные, target_neg: теплые


[CLS] Слон -  [MASK] животное. [SEP] 
[CLS] Слон - не  [MASK] животное. [SEP] 
Predictions aff:
- это
- многозначное
- ночное
- морское
- живое
Predictions neg:
- одно
- домашнее
- просто
- животное
- настоящее
Predictions aff: ['это', 'многозначное', 'ночное', 'морское', 'живое']
Predictions neg: ['одно', 'домашнее', 'просто', 'животное', 'настоящее']
FAIL AFF: predicted != expected
FAIL NEG: predicted != expected
target_aff: большое, target_neg: маленькое


[CLS] Колибри - самые  [MASK] птицы. [SEP] 
[CLS] Колибри - не самые  

In [0]:
accuracy = predict_on_rus_neg(rubert_tokenizer, rubert_model, df_rus_neg, 10)
print("ACCURACY = {}".format(accuracy))

[CLS] В Якутии зимы очень  [MASK] . [SEP] 
[CLS] В Якутии зимы не очень  [MASK] . [SEP] 
Predictions aff:
- холодные
- суровые
- мягкие
- теплые
- тёплые
- короткие
- тепло
- сухие
- тяжелые
- длинные
Predictions neg:
- холодные
- суровые
- теплые
- тепло
- мягкие
- тёплые
- сильны
- сухие
- много
- хорошие
Predictions aff: ['холодные', 'суровые', 'мягкие', 'теплые', 'тёплые', 'короткие', 'тепло', 'сухие', 'тяжелые', 'длинные']
Predictions neg: ['холодные', 'суровые', 'теплые', 'тепло', 'мягкие', 'тёплые', 'сильны', 'сухие', 'много', 'хорошие']
PASS AFF: predicted == expected
PASS NEG: predicted == expected
target_aff: холодные, target_neg: теплые


[CLS] Слон -  [MASK] животное. [SEP] 
[CLS] Слон - не  [MASK] животное. [SEP] 
Predictions aff:
- это
- многозначное
- ночное
- морское
- живое
- домашнее
- редкое
- огромное
- животное
- другое
Predictions neg:
- одно
- домашнее
- просто
- животное
- настоящее
- единственное
- женское
- обычное
- это
- главное
Predictions aff: ['это', 'мно

# RUS-ROLE-24

In [0]:
df_RUS_ROLE24 =pd.read_excel(os.path.join("datasets", "RUS-ROLE-24.xlsx"),  encoding="cp866")
df_RUS_ROLE24.head()

,context,expected
0,"Полицейский выяснил, что маньяка жертва * .",встретила
1,"Полицейский выяснил, что жертву маньяк * .",убил
2,"Судья выяснил, какого продавца грабитель * .",ограбил
3,"Судья выяснил, какого грабителя продавец * .",опознал
4,"Директор узнал, какого ученика учитель * .",учит | учил


In [0]:
def add_special_tokens(context):
  context = context.replace("*", MASK)
  context = CLS + context + SEP
  return context

In [0]:
for i in range(0, df_RUS_ROLE24.shape[0]):
  df_RUS_ROLE24.at[i, "context"] = add_special_tokens(df_RUS_ROLE24.iloc[i]["context"])
  print("{}".format(df_RUS_ROLE24.iloc[i]["context"]))

[CLS] Полицейский выяснил, что маньяка жертва  [MASK] . [SEP] 
[CLS] Полицейский выяснил, что жертву маньяк  [MASK] . [SEP] 
[CLS] Судья выяснил, какого продавца грабитель  [MASK] . [SEP] 
[CLS] Судья выяснил, какого грабителя продавец  [MASK] . [SEP] 
[CLS] Директор узнал, какого ученика учитель  [MASK] . [SEP] 
[CLS] Директор узнал, какого учителя ученика  [MASK] . [SEP] 
[CLS] Продавец узнал по штрих-коду, какой товар покупатель  [MASK] . [SEP] 
[CLS] Покупатель узнал по штрих-коду, какой товар продавец  [MASK] . [SEP] 
[CLS] Игрок во время игры  [MASK] вратарю 5 голов. [SEP] 
[CLS] Вратарь во время игры  [MASK] 5 голов от игрока. [SEP] 
[CLS] Ребенок  [MASK] у мамы погулять. [SEP] 
[CLS] Мама  [MASK] ребенка погулять. [SEP] 
[CLS] Ребенок хочет пить. Мама  [MASK] ребенку воды. [SEP] 
[CLS] Ребенок хочет пить. Ребенок  [MASK] у мамы воды. [SEP] 
[CLS] Чтобы замять дело, мужчина  [MASK] взятку прокурору. [SEP] 
[CLS] Чтобы замять дело, прокурор  [MASK] взятку у мужчины. [SEP] 
[CLS] 

In [0]:
def predict_word(context, tokenizer, model, k=1):
  tokenized_text = tokenizer.tokenize(context)
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

  segments_ids = [0] * len(tokenized_text)
  #print(tokenized_text)
  masked_index = tokenized_text.index('[MASK]') 

  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])
  model.eval()
  with torch.no_grad():
    predictions = model(tokens_tensor, segments_tensors)
  predicted_index = np.argmax(predictions[0][0,masked_index,:])
  predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
  res = np.argsort(predictions[0][0, masked_index].numpy())[::-1][:k]
  return res

In [0]:
def predict_on_ROLE_24(tokenizer, model, df_RUS_ROLE24, k=1):
  count = 0
  passed = 0
  failed = 0
  for i in range(0, df_RUS_ROLE24.shape[0]):
    print("{}".format(df_RUS_ROLE24.iloc[i]["context"]))
    predicted_ids = predict_word(df_RUS_ROLE24.iloc[i]["context"], tokenizer, model, k)
    predicted_words = [tokenizer.convert_ids_to_tokens([word])[0] for word in predicted_ids]
    predicted_words = [word.replace('▁', '') for word in predicted_words]
    #print('Predictions: {}'.format(predicted_words))
    print('Predictions:')
    for pred in predicted_words:
      print('- {}'.format(pred))
    exps = df_RUS_ROLE24.iloc[i]["expected"].split("|")

    detected = False
    for exp in exps:
      if check_predicted_words(predicted_words, exp):
        detected = True
        break
    if detected:
      passed += 1
      print("PASS: predicted == expected")
    else:
      failed += 1
      print("FAIL: predicted != expected") 
    count += 1
    print("\n")
  accuracy = (float)(passed / count) 
  print("------------------------------------")
  print("ALL: {} | PASSED: {} | FAILED: {}".format(count, passed, failed))
  print("------------------------------------")
  return round(accuracy, 2)

In [0]:
from transformers import AutoTokenizer, AutoModelWithLMHead

weights = "DrMatters/rubert_cased"
rubert_tokenizer = AutoTokenizer.from_pretrained(weights)
rubert_model = AutoModelWithLMHead.from_pretrained(weights)

In [0]:
accuracy = predict_on_ROLE_24(rubert_tokenizer, rubert_model, df_RUS_ROLE24, 1)
print("ACCURACY = {}".format(accuracy))

[CLS] Полицейский выяснил, что маньяка жертва  [MASK] . [SEP] 
Predictions:
- убила
FAIL: predicted != expected


[CLS] Полицейский выяснил, что жертву маньяк  [MASK] . [SEP] 
Predictions:
- убил
PASS: predicted == expected


[CLS] Судья выяснил, какого продавца грабитель  [MASK] . [SEP] 
Predictions:
- застрелил
FAIL: predicted != expected


[CLS] Судья выяснил, какого грабителя продавец  [MASK] . [SEP] 
Predictions:
- задержал
FAIL: predicted != expected


[CLS] Директор узнал, какого ученика учитель  [MASK] . [SEP] 
Predictions:
- избил
FAIL: predicted != expected


[CLS] Директор узнал, какого учителя ученика  [MASK] . [SEP] 
Predictions:
- выгнали
FAIL: predicted != expected


[CLS] Продавец узнал по штрих-коду, какой товар покупатель  [MASK] . [SEP] 
Predictions:
- ##нице
FAIL: predicted != expected


[CLS] Покупатель узнал по штрих-коду, какой товар продавец  [MASK] . [SEP] 
Predictions:
- приобрел
FAIL: predicted != expected


[CLS] Игрок во время игры  [MASK] вратарю 5 голов. 

In [0]:
accuracy = predict_on_ROLE_24(rubert_tokenizer, rubert_model, df_RUS_ROLE24, 3)
print("ACCURACY = {}".format(accuracy))

[CLS] Полицейский выяснил, что маньяка жертва  [MASK] . [SEP] 
Predictions:
- убила
- преследовала
- видела
FAIL: predicted != expected


[CLS] Полицейский выяснил, что жертву маньяк  [MASK] . [SEP] 
Predictions:
- убил
- изнасиловал
- задушил
PASS: predicted == expected


[CLS] Судья выяснил, какого продавца грабитель  [MASK] . [SEP] 
Predictions:
- застрелил
- обманул
- убил
FAIL: predicted != expected


[CLS] Судья выяснил, какого грабителя продавец  [MASK] . [SEP] 
Predictions:
- задержал
- нанял
- нашел
FAIL: predicted != expected


[CLS] Директор узнал, какого ученика учитель  [MASK] . [SEP] 
Predictions:
- избил
- бросил
- убил
FAIL: predicted != expected


[CLS] Директор узнал, какого учителя ученика  [MASK] . [SEP] 
Predictions:
- выгнали
- нет
- зовут
FAIL: predicted != expected


[CLS] Продавец узнал по штрих-коду, какой товар покупатель  [MASK] . [SEP] 
Predictions:
- ##нице
- ##у
- ##е
FAIL: predicted != expected


[CLS] Покупатель узнал по штрих-коду, какой товар продавец

In [0]:
accuracy = predict_on_ROLE_24(rubert_tokenizer, rubert_model, df_RUS_ROLE24, 5)
print("ACCURACY = {}".format(accuracy))

[CLS] Полицейский выяснил, что маньяка жертва  [MASK] . [SEP] 
Predictions:
- убила
- преследовала
- видела
- убийства
- убийцы
FAIL: predicted != expected


[CLS] Полицейский выяснил, что жертву маньяк  [MASK] . [SEP] 
Predictions:
- убил
- изнасиловал
- задушил
- убивал
- избил
PASS: predicted == expected


[CLS] Судья выяснил, какого продавца грабитель  [MASK] . [SEP] 
Predictions:
- застрелил
- обманул
- убил
- нашел
- нанял
FAIL: predicted != expected


[CLS] Судья выяснил, какого грабителя продавец  [MASK] . [SEP] 
Predictions:
- задержал
- нанял
- нашел
- видел
- поймал
FAIL: predicted != expected


[CLS] Директор узнал, какого ученика учитель  [MASK] . [SEP] 
Predictions:
- избил
- бросил
- убил
- знает
- учил
FAIL: predicted != expected


[CLS] Директор узнал, какого учителя ученика  [MASK] . [SEP] 
Predictions:
- выгнали
- нет
- зовут
- убили
- убил
FAIL: predicted != expected


[CLS] Продавец узнал по штрих-коду, какой товар покупатель  [MASK] . [SEP] 
Predictions:
- ##нице


In [0]:
accuracy = predict_on_ROLE_24(rubert_tokenizer, rubert_model, df_RUS_ROLE24, 10)
print("ACCURACY = {}".format(accuracy))

[CLS] Полицейский выяснил, что маньяка жертва  [MASK] . [SEP] 
Predictions:
- убила
- преследовала
- видела
- убийства
- убийцы
- убил
- знала
- убили
- выбросил
- убивает
FAIL: predicted != expected


[CLS] Полицейский выяснил, что жертву маньяк  [MASK] . [SEP] 
Predictions:
- убил
- изнасиловал
- задушил
- убивал
- избил
- похитил
- бросил
- застрелил
- убивает
- расстрелял
PASS: predicted == expected


[CLS] Судья выяснил, какого продавца грабитель  [MASK] . [SEP] 
Predictions:
- застрелил
- обманул
- убил
- нашел
- нанял
- выбрал
- ранил
- ударил
- искал
- избил
FAIL: predicted != expected


[CLS] Судья выяснил, какого грабителя продавец  [MASK] . [SEP] 
Predictions:
- задержал
- нанял
- нашел
- видел
- поймал
- испугался
- застрелил
- ранил
- ищет
- обманул
FAIL: predicted != expected


[CLS] Директор узнал, какого ученика учитель  [MASK] . [SEP] 
Predictions:
- избил
- бросил
- убил
- знает
- учил
- узнал
- знал
- видел
- застрелил
- избивает
FAIL: predicted != expected


[CLS] Д